In [7]:
# ==========================================
# VerimGören — Tek Nokta Rapor (birleştirilmiş, sade)
# ==========================================
from pathlib import Path
from typing import Dict
import pandas as pd
import numpy as np

# Opsiyonel bağımlılıklar (yüklü değilse None olur ve ilgili kısım atlanır)
try:
    import rasterio
except Exception:
    rasterio = None

try:
    import pyodbc
except Exception:
    pyodbc = None

# -------------------------------------------------
# Yollar — KENDİ YOLLARINLA GÜNCELLE
# -------------------------------------------------
HWSD_BASE = Path(r"C:/Users/ataka/Desktop/MEHMET/VerimGören/notebooks/hwsd_data")
HWSD_MDB  = HWSD_BASE / "HWSD.mdb"     # Access veritabanı
HWSD_RAS  = HWSD_BASE / "hwsd.bil"     # MU_GLOBAL raster (bil)

CSV_PATH   = Path(r"C:..\notebooks\data\climate\merged_climate_data.csv")   # İklim (0.5° grid)
ELEV_PATH  = Path(r"C:..\data\processed\srtm_turkiye_cropped.tif")          # Rakım (opsiyonel)
LIGHT_PATH = Path(r"C:..\data\processed\viirs_light_2024_turkey.tif")       # Gece ışığı (opsiyonel)

# -------------------------------------------------
# Değişken Anlamları & Kategoriler (TR meta)
# -------------------------------------------------
CATEGORY_ORDER = ["Konum", "İklim", "Arazi", "Gece Işığı", "Toprak", "Özet"]

def category_of(key: str) -> str:
    k = key.upper()
    if k in {"LAT", "LON", "LATITUDE", "LONGITUDE"}:
        return "Konum"
    if k in {"ELEVATION_M"}:
        return "Arazi"
    if k in {"NIGHT_LIGHT", "VIIRS_NTL"}:
        return "Gece Işığı"
    if (
        "_GRP" in k
        or k in {
            "T2M","T2M_MAX","T2M_MIN","T2M_RANGE","T2MDEW","T2MWET",
            "RH2M","QV2M","TQV","PS","SLP","WS2M","WS2M_MAX","WD2M",
            "PRECTOTCORR","TS","TO3","ALLSKY_SFC_SW_DWN","ALLSKY_SFC_PAR_TOT",
            "CLRSKY_SFC_SW_DWN","CLOUD_AMT","CLOUD_AMT_DAY","CLOUD_AMT_NIGHT","CLRSKY_DAYS"
        }
    ):
        return "İklim"
    if k in {
        "FAO90_DESC","T_USDA_TEX_DESC","S_USDA_TEX_DESC","T_TEXTURE_DESC",
        "T_SAND","T_SILT","T_CLAY","S_SAND","S_SILT","S_CLAY",
        "T_PH_H2O","S_PH_H2O","T_OC","S_OC","T_CEC_SOIL","S_CEC_SOIL",
        "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB",
        "T_CACO3","S_CACO3","AWC_MM_PER_M","DRAINAGE_DESC","T_ECE","S_ECE","T_ESP","S_ESP",
        "MU_GLOBAL"
    }:
        return "Toprak"
    if k in {"DISTANCE_KM","DISTANCE_KM_IDW"}:
        return "Özet"
    return "Özet"

VAR_META: Dict[str, Dict[str, str]] = {
    # Konum
    "latitude":   {"title_tr": "Enlem",   "meaning": "Coğrafi enlem", "unit": "°"},
    "longitude":  {"title_tr": "Boylam",  "meaning": "Coğrafi boylam", "unit": "°"},
    "LAT":        {"title_tr": "Enlem",   "meaning": "Coğrafi enlem (HWSD)", "unit": "°"},
    "LON":        {"title_tr": "Boylam",  "meaning": "Coğrafi boylam (HWSD)", "unit": "°"},

    # İklim – radyasyon/par
    "ALLSKY_SFC_PAR_TOT":     {"title_tr":"PAR (tümü)", "meaning":"Fotosentetik aktif radyasyon (tüm gökyüzü)", "unit":"MJ/m²/gün"},
    "ALLSKY_SFC_PAR_TOT_GRP1":{"title_tr":"PAR (tümü)", "meaning":"Fotosentetik aktif radyasyon (tüm gökyüzü)", "unit":"MJ/m²/gün"},
    "ALLSKY_SFC_SW_DWN":      {"title_tr":"Kısa dalga (tümü)", "meaning":"Yüzeye inen kısa dalga güneş radyasyonu (tüm gökyüzü)", "unit":"kWh/m²/gün"},
    "ALLSKY_SFC_SW_DWN_GRP1": {"title_tr":"Kısa dalga (tümü)", "meaning":"Yüzeye inen kısa dalga güneş radyasyonu (tüm gökyüzü)", "unit":"kWh/m²/gün"},
    "CLRSKY_SFC_SW_DWN":      {"title_tr":"Kısa dalga (açık gök)", "meaning":"Bulutsuz gökyüzü kısa dalga radyasyon", "unit":"kWh/m²/gün"},
    "CLRSKY_SFC_SW_DWN_GRP1": {"title_tr":"Kısa dalga (açık gök)", "meaning":"Bulutsuz gökyüzü kısa dalga radyasyon", "unit":"kWh/m²/gün"},
    "CLRSKY_DAYS":            {"title_tr":"Açık gün sayısı", "meaning":"Ay içindeki açık gün sayısı", "unit":"gün/ay"},
    "CLRSKY_DAYS_GRP1":       {"title_tr":"Açık gün sayısı", "meaning":"Ay içindeki açık gün sayısı", "unit":"gün/ay"},

    # İklim – bulut/atmosfer
    "CLOUD_AMT":              {"title_tr":"Bulutluluk", "meaning":"Ortalama bulutluluk oranı", "unit":"%"},
    "CLOUD_AMT_GRP1":         {"title_tr":"Bulutluluk", "meaning":"Ortalama bulutluluk oranı", "unit":"%"},
    "CLOUD_AMT_DAY":          {"title_tr":"Bulutluluk (gündüz)", "meaning":"Gündüz ortalama bulutluluk", "unit":"%"},
    "CLOUD_AMT_DAY_GRP1":     {"title_tr":"Bulutluluk (gündüz)", "meaning":"Gündüz ortalama bulutluluk", "unit":"%"},
    "CLOUD_AMT_NIGHT":        {"title_tr":"Bulutluluk (gece)", "meaning":"Gece ortalama bulutluluk", "unit":"%"},
    "CLOUD_AMT_NIGHT_GRP1":   {"title_tr":"Bulutluluk (gece)", "meaning":"Gece ortalama bulutluluk", "unit":"%"},

    # İklim – nem/sıcaklık
    "QV2M":        {"title_tr":"Özgül nem (2 m)", "meaning":"2 m'de su buharı miktarı", "unit":"g/kg"},
    "QV2M_GRP2":   {"title_tr":"Özgül nem (2 m)", "meaning":"2 m'de su buharı miktarı", "unit":"g/kg"},
    "RH2M":        {"title_tr":"Bağıl nem (2 m)", "meaning":"2 m'de bağıl nem", "unit":"%"},
    "RH2M_GRP2":   {"title_tr":"Bağıl nem (2 m)", "meaning":"2 m'de bağıl nem", "unit":"%"},
    "T2M":         {"title_tr":"Sıcaklık (2 m, ort.)", "meaning":"2 m'de ortalama hava sıcaklığı", "unit":"°C"},
    "T2M_GRP2":    {"title_tr":"Sıcaklık (2 m, ort.)", "meaning":"2 m'de ortalama hava sıcaklığı", "unit":"°C"},
    "T2M_MAX":     {"title_tr":"Maks. sıcaklık", "meaning":"Günlük maksimum hava sıcaklığı", "unit":"°C"},
    "T2M_MAX_GRP2":{"title_tr":"Maks. sıcaklık", "meaning":"Günlük maksimum hava sıcaklığı", "unit":"°C"},
    "T2M_MIN":     {"title_tr":"Min. sıcaklık", "meaning":"Günlük minimum hava sıcaklığı", "unit":"°C"},
    "T2M_MIN_GRP2":{"title_tr":"Min. sıcaklık", "meaning":"Günlük minimum hava sıcaklığı", "unit":"°C"},
    "T2M_RANGE":   {"title_tr":"Günlük sıcaklık aralığı", "meaning":"Maks–min farkı", "unit":"°C"},
    "T2M_RANGE_GRP2":{"title_tr":"Günlük sıcaklık aralığı", "meaning":"Maks–min farkı", "unit":"°C"},
    "T2MDEW":      {"title_tr":"Çiy noktası", "meaning":"Doygunluk sıcaklığı", "unit":"°C"},
    "T2MDEW_GRP2": {"title_tr":"Çiy noktası", "meaning":"Doygunluk sıcaklığı", "unit":"°C"},
    "T2MWET":      {"title_tr":"Yaş termometre", "meaning":"Buharlaşma etkili sıcaklık", "unit":"°C"},
    "T2MWET_GRP2": {"title_tr":"Yaş termometre", "meaning":"Buharlaşma etkili sıcaklık", "unit":"°C"},
    "TQV":         {"title_tr":"Kolon su buharı", "meaning":"Atmosfer kolonundaki toplam su buharı", "unit":"kg/m²"},
    "TQV_GRP2":    {"title_tr":"Kolon su buharı", "meaning":"Atmosfer kolonundaki toplam su buharı", "unit":"kg/m²"},

    # İklim – basınç/rüzgar/yağış/ozon/yüzey
    "PS":          {"title_tr":"Yüzey basıncı", "meaning":"Yüzeyde atmosfer basıncı", "unit":"kPa"},
    "PS_GRP3":     {"title_tr":"Yüzey basıncı", "meaning":"Yüzeyde atmosfer basıncı", "unit":"kPa"},
    "SLP":         {"title_tr":"Denize indirgenmiş basınç", "meaning":"MSL'e indirgenmiş basınç", "unit":"kPa"},
    "SLP_GRP3":    {"title_tr":"Denize indirgenmiş basınç", "meaning":"MSL'e indirgenmiş basınç", "unit":"kPa"},
    "WD2M":        {"title_tr":"Rüzgar yönü (2 m)", "meaning":"2 m rüzgar yönü", "unit":"°"},
    "WD2M_GRP3":   {"title_tr":"Rüzgar yönü (2 m)", "meaning":"2 m rüzgar yönü", "unit":"°"},
    "WS2M":        {"title_tr":"Rüzgar hızı (2 m)", "meaning":"2 m ortalama rüzgar hızı", "unit":"m/s"},
    "WS2M_GRP3":   {"title_tr":"Rüzgar hızı (2 m)", "meaning":"2 m ortalama rüzgar hızı", "unit":"m/s"},
    "WS2M_MAX":    {"title_tr":"Maks. rüzgar (2 m)", "meaning":"2 m maksimum rüzgar hızı", "unit":"m/s"},
    "WS2M_MAX_GRP3":{"title_tr":"Maks. rüzgar (2 m)", "meaning":"2 m maksimum rüzgar hızı", "unit":"m/s"},
    "PRECTOTCORR": {"title_tr":"Toplam yağış (düz.)", "meaning":"Düzeltilmiş toplam yağış", "unit":"mm/gün"},
    "PRECTOTCORR_GRP4":{"title_tr":"Toplam yağış (düz.)", "meaning":"Düzeltilmiş toplam yağış", "unit":"mm/gün"},
    "TO3":         {"title_tr":"Toplam ozon", "meaning":"Toplam ozon kolonu", "unit":"DU"},
    "TO3_GRP4":    {"title_tr":"Toplam ozon", "meaning":"Toplam ozon kolonu", "unit":"DU"},
    "TS":          {"title_tr":"Yüzey sıcaklığı", "meaning":"Zemin sıcaklığı", "unit":"°C"},
    "TS_GRP4":     {"title_tr":"Yüzey sıcaklığı", "meaning":"Zemin sıcaklığı", "unit":"°C"},

    # Arazi & Gece ışığı
    "ELEVATION_M": {"title_tr":"Rakım", "meaning":"Deniz seviyesinden yükseklik", "unit":"m"},
    "NIGHT_LIGHT": {"title_tr":"Gece ışığı", "meaning":"Yapay aydınlatma yoğunluğu (VIIRS)", "unit":"-"},

    # Toprak – sınıf & doku
    "FAO90_DESC":      {"title_tr":"FAO-90 sınıfı", "meaning":"FAO-90 toprak sınıfı açıklaması", "unit":"-"},
    "T_USDA_TEX_DESC": {"title_tr":"Üst USDA doku", "meaning":"Üst toprak USDA doku sınıfı", "unit":"-"},
    "S_USDA_TEX_DESC": {"title_tr":"Alt USDA doku", "meaning":"Alt toprak USDA doku sınıfı", "unit":"-"},
    "T_TEXTURE_DESC":  {"title_tr":"Üst doku (coarse/medium/fine)", "meaning":"Üst toprak doku açıklaması", "unit":"-"},

    # Toprak – fiziksel yüzde
    "T_SAND": {"title_tr":"Kum (üst)",  "meaning":"Üst toprak kum oranı", "unit":"%"},
    "T_SILT": {"title_tr":"Silt (üst)", "meaning":"Üst toprak silt oranı", "unit":"%"},
    "T_CLAY": {"title_tr":"Kil (üst)",  "meaning":"Üst toprak kil oranı", "unit":"%"},
    "S_SAND": {"title_tr":"Kum (alt)",  "meaning":"Alt toprak kum oranı", "unit":"%"},
    "S_SILT": {"title_tr":"Silt (alt)", "meaning":"Alt toprak silt oranı", "unit":"%"},
    "S_CLAY": {"title_tr":"Kil (alt)",  "meaning":"Alt toprak kil oranı", "unit":"%"},

    # Toprak – kimya
    "T_PH_H2O":    {"title_tr":"pH (üst)", "meaning":"Üst toprak pH (H₂O)", "unit":"-"},
    "S_PH_H2O":    {"title_tr":"pH (alt)", "meaning":"Alt toprak pH (H₂O)", "unit":"-"},
    "T_OC":        {"title_tr":"Organik C (üst)", "meaning":"Üst toprak organik karbon", "unit":"%"},
    "S_OC":        {"title_tr":"Organik C (alt)", "meaning":"Alt toprak organik karbon", "unit":"%"},
    "T_CEC_SOIL":  {"title_tr":"KDK (üst)", "meaning":"Üst toprak katyon değişim kapasitesi", "unit":"cmol(+)/kg"},
    "S_CEC_SOIL":  {"title_tr":"KDK (alt)", "meaning":"Alt toprak katyon değişim kapasitesi", "unit":"cmol(+)/kg"},
    "T_CEC_CLAY":  {"title_tr":"KDK (kil, üst)", "meaning":"Üst toprak kil fraksiyonu KDK", "unit":"cmol(+)/kg"},
    "S_CEC_CLAY":  {"title_tr":"KDK (kil, alt)", "meaning":"Alt toprak kil fraksiyonu KDK", "unit":"cmol(+)/kg"},
    "T_BS":        {"title_tr":"Baz doygunluğu (üst)", "meaning":"Üst toprak baz doygunluğu", "unit":"%"},
    "S_BS":        {"title_tr":"Baz doygunluğu (alt)", "meaning":"Alt toprak baz doygunluğu", "unit":"%"},
    "T_TEB":       {"title_tr":"Toplam değ. baz (üst)", "meaning":"Üst toprak toplam değişebilir bazlar", "unit":"cmol(+)/kg"},
    "S_TEB":       {"title_tr":"Toplam değ. baz (alt)", "meaning":"Alt toprak toplam değişebilir bazlar", "unit":"cmol(+)/kg"},
    "T_CACO3":     {"title_tr":"Kireç CaCO₃ (üst)", "meaning":"Üst toprak kireç", "unit":"%"},
    "S_CACO3":     {"title_tr":"Kireç CaCO₃ (alt)", "meaning":"Alt toprak kireç", "unit":"%"},
    "T_ECE":       {"title_tr":"EC (üst)", "meaning":"Üst toprak elektriksel iletkenlik", "unit":"dS/m"},
    "S_ECE":       {"title_tr":"EC (alt)", "meaning":"Alt toprak elektriksel iletkenlik", "unit":"dS/m"},
    "T_ESP":       {"title_tr":"ESP (üst)", "meaning":"Üst toprak değişebilir sodyum yüzdesi", "unit":"%"},
    "S_ESP":       {"title_tr":"ESP (alt)", "meaning":"Alt toprak değişebilir sodyum yüzdesi", "unit":"%"},

    # Toprak – su/drenaj/harita birimi
    "AWC_MM_PER_M":   {"title_tr":"Kullanılabilir su (AWC)", "meaning":"Kök bölgesi kullanılabilir su kapasitesi", "unit":"mm/m"},
    "DRAINAGE_DESC":  {"title_tr":"Drenaj", "meaning":"Drenaj sınıfı açıklaması", "unit":"-"},
    "MU_GLOBAL":      {"title_tr":"Harita birimi (MU)", "meaning":"HWSD harita birimi kodu", "unit":"-"},

    # Özet
    "DISTANCE_KM":     {"title_tr":"Uzaklık (iklim pikseli)", "meaning":"İklim verisindeki en yakın hücre mesafesi", "unit":"km"},
    "DISTANCE_KM_IDW": {"title_tr":"Uzaklık (IDW etkin)", "meaning":"Ters-mesafe ağırlıklı etkin uzaklık", "unit":"km"},
}

def describe_var(key: str) -> Dict[str, str]:
    meta = VAR_META.get(key) or VAR_META.get(key.upper())
    out = {
        "key": key,
        "title_tr": meta["title_tr"] if meta else key,
        "meaning":  meta["meaning"]  if meta else "(Açıklama bulunamadı)",
        "unit":     meta["unit"]     if meta else "-",
        "category": category_of(key),
    }
    return out

# -------------------------------------------------
# HWSD: Access & raster yordamları
# -------------------------------------------------
def _read_table_access(table_name, mdb_path=HWSD_MDB) -> pd.DataFrame:
    mdb_path = Path(mdb_path).resolve()
    if not mdb_path.exists():
        raise FileNotFoundError(f"HWSD.mdb bulunamadı: {mdb_path}")
    if pyodbc is None:
        raise ImportError("pyodbc bulunamadı. Microsoft Access sürücüsü gereklidir.")
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={str(mdb_path).replace('\\', '/')};"
    cn = pyodbc.connect(conn_str)
    try:
        return pd.read_sql(f"SELECT * FROM {table_name}", cn)
    finally:
        cn.close()

def _normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = [c.strip().upper() for c in df.columns]
    return out

def _get_code_desc_cols(df: pd.DataFrame):
    cols = set(df.columns)
    code = "CODE" if "CODE" in cols else None
    desc = "DESCRIPTION" if "DESCRIPTION" in cols else ("VALUE" if "VALUE" in cols else None)
    return code, desc

def _mu_from_latlon(lat: float, lon: float, raster_path=HWSD_RAS) -> int:
    rp = Path(raster_path).resolve()
    if rasterio is None:
        raise ImportError("rasterio bulunamadı. MU_GLOBAL okumak için rasterio gerekir.")
    if not rp.exists():
        raise FileNotFoundError(f"HWSD raster bulunamadı: {rp}")
    with rasterio.open(rp) as src:
        r, c = src.index(lon, lat)  # WGS84 varsayımı
        mu = int(src.read(1)[r, c])
    if mu <= 0:
        raise ValueError(f"Geçersiz MU_GLOBAL={mu}")
    return mu

_HWSD_DATA = None
_DOM = None

def _load_hwsd_dom_table():
    global _HWSD_DATA, _DOM
    if _DOM is not None:
        return _DOM

    hwsd = _normalize_cols(_read_table_access("HWSD_DATA"))

    def _safe(name):
        try:
            return _normalize_cols(_read_table_access(name))
        except Exception:
            return None

    tex   = _safe("D_TEXTURE")
    utex  = _safe("D_USDA_TEX_CLASS")
    awc   = _safe("D_AWC")
    drn   = _safe("D_DRAINAGE")
    sym90 = _safe("D_SYMBOL90")

    def _lut(df, _code, _desc, out_code, out_desc):
        if isinstance(df, pd.DataFrame):
            c, d = _get_code_desc_cols(df)
            if c and d:
                return df.rename(columns={c: out_code, d: out_desc})[[out_code, out_desc]]
        return None

    tex_lut   = _lut(tex,  None, None, "T_TEXTURE", "T_TEXTURE_DESC")
    usda_lutT = _lut(utex, None, None, "T_USDA_TEX_CLASS", "T_USDA_TEX_DESC")
    usda_lutS = _lut(utex, None, None, "S_USDA_TEX_CLASS", "S_USDA_TEX_DESC")
    awc_lut   = _lut(awc,  None, None, "AWC_CLASS", "AWC_MM_PER_M")
    drn_lut   = _lut(drn,  None, None, "DRAINAGE", "DRAINAGE_DESC")
    sym90_lut = _lut(sym90,None, None, "SU_CODE90", "FAO90_DESC")

    df = hwsd.copy()
    if tex_lut   is not None and "T_TEXTURE"        in df: df = df.merge(tex_lut,   on="T_TEXTURE",        how="left")
    if usda_lutT is not None and "T_USDA_TEX_CLASS" in df: df = df.merge(usda_lutT, on="T_USDA_TEX_CLASS", how="left")
    if usda_lutS is not None and "S_USDA_TEX_CLASS" in df: df = df.merge(usda_lutS, on="S_USDA_TEX_CLASS", how="left")
    if awc_lut   is not None and "AWC_CLASS"        in df: df = df.merge(awc_lut,   on="AWC_CLASS",        how="left")
    if drn_lut   is not None and "DRAINAGE"         in df: df = df.merge(drn_lut,   on="DRAINAGE",         how="left")
    if sym90_lut is not None and "SU_CODE90"        in df: df = df.merge(sym90_lut, on="SU_CODE90",        how="left")

    # sayısal alanları float'a çevir
    num_cols = [
        "AWC_MM_PER_M","T_PH_H2O","S_PH_H2O","T_OC","S_OC",
        "T_CLAY","T_SILT","T_SAND","S_CLAY","S_SILT","S_SAND",
        "T_ECE","S_ECE","T_ESP","S_ESP","T_CEC_SOIL","S_CEC_SOIL",
        "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB","T_CACO3","S_CACO3",
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")

    # aynı MU için baskın bileşen
    dom = (df.sort_values(["MU_GLOBAL","SEQ","SHARE"], ascending=[True,True,False])
             .groupby("MU_GLOBAL", as_index=False)
             .first())

    _HWSD_DATA = df
    _DOM = dom
    return dom

def hwsd_point_row(lat: float, lon: float) -> pd.Series:
    mu = _mu_from_latlon(lat, lon, raster_path=HWSD_RAS)
    dom = _load_hwsd_dom_table()
    row = dom.loc[dom["MU_GLOBAL"] == mu]
    if row.empty:
        raise LookupError(f"MU_GLOBAL={mu} için kayıt yok.")
    s = row.iloc[0].copy()
    s["LAT"], s["LON"] = lat, lon
    return s

def hwsd_point_report(lat: float, lon: float, columns="all"):
    s = hwsd_point_row(lat, lon)
    txt = ""
    if columns == "all":
        return txt, s
    else:
        return txt, s[columns]

# -------------------------------------------------
# Coğrafi yardımcılar & raster örnekleme
# -------------------------------------------------
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def sample_raster(path: Path, lon: float, lat: float):
    try:
        if rasterio is None or not path.exists():
            return None
        with rasterio.open(path) as ds:
            r, c = ds.index(lon, lat)  # WGS84 varsayımı
            val = ds.read(1)[r, c]
            if ds.nodata is not None and val == ds.nodata:
                return None
            return float(val)
    except Exception:
        return None

def load_climate_nearest(csv_path: Path, lat: float, lon: float):
    df = pd.read_csv(csv_path)
    if not {"latitude","longitude"}.issubset(df.columns):
        raise ValueError("CSV'de 'latitude' ve 'longitude' sütunları yok.")
    dist = haversine(lat, lon, df["latitude"].values, df["longitude"].values)
    i = int(np.argmin(dist))
    row = df.iloc[i].to_dict()
    row["DISTANCE_KM"] = float(dist[i])
    return row

def try_load_soil(lat: float, lon: float):
    try:
        _, row = hwsd_point_report(lat, lon, columns="all")
        return row.to_dict()
    except Exception:
        return None

def format_value(v):
    if v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v))):
        return "-"
    try:
        f = float(v)
        if abs(f - round(f)) < 1e-9:
            return f"{int(round(f))}"
        return f"{f:.2f}"
    except Exception:
        return str(v)

# -------------------------------------------------
# Tek Nokta Raporu
# -------------------------------------------------
def run_point_report(lat: float, lon: float):
    blocks = {k: [] for k in CATEGORY_ORDER}

    # 1) İklim (en yakın hücre)
    clim = load_climate_nearest(CSV_PATH, lat, lon)
    for k, v in clim.items():
        cat = category_of(k)
        meta = describe_var(k)
        unit = meta["unit"]
        suffix = f" {unit}" if unit and unit != "-" else ""
        blocks[cat].append(f"- {meta['title_tr']} ({k}): {format_value(v)}{suffix}")

    # 2) Rakım / Gece Işığı (opsiyonel)
    elev = sample_raster(ELEV_PATH, lon, lat)
    if elev is not None:
        meta = describe_var("ELEVATION_M")
        blocks["Arazi"].append(f"- {meta['title_tr']} (ELEVATION_M): {format_value(elev)} {meta['unit']}")
    light = sample_raster(LIGHT_PATH, lon, lat)
    if light is not None:
        meta = describe_var("NIGHT_LIGHT")
        unit_sfx = f" {meta['unit']}" if meta["unit"] and meta["unit"] != "-" else ""
        blocks["Gece Işığı"].append(f"- {meta['title_tr']} (NIGHT_LIGHT): {format_value(light)}{unit_sfx}")

    # 3) Toprak (HWSD)
    soil = try_load_soil(lat, lon)
    if soil:
        for k, v in soil.items():
            meta = describe_var(k)
            cat = category_of(k)
            unit = meta["unit"]
            suffix = f" {unit}" if unit and unit != "-" else ""
            blocks[cat].append(f"- {meta['title_tr']} ({k}): {format_value(v)}{suffix}")

    # ---- Çıktı ----
    print(f"📍 Konum: {lat:.5f}, {lon:.5f}\n")
    icons = {"Konum":"📌","İklim":"🌦️","Arazi":"⛰️","Gece Işığı":"🌃","Toprak":"🌱","Özet":"🧾"}
    for cat in CATEGORY_ORDER:
        if blocks.get(cat):
            print(f"{icons.get(cat,'•')} {cat}")
            for line in blocks[cat]:
                print(line)
            print()

# -------------------------------------------------
# Örnek kullanım
# -------------------------------------------------
# -------------------------------------------------
# Örnek kullanım / CLI
# -------------------------------------------------
# Küçük başlatıcı: /start [lat lon] veya Google Maps linki
import re
import sys
import argparse

LATLON_RE = re.compile(r'(-?\d+(?:\.\d+)?)\s*[, ]\s*(-?\d+(?:\.\d+)?)')

def parse_latlon(s: str):
    """
    Şu formatları yakalar:
    - '41.2397, 41.9156'  veya  '41.2397 41.9156'
    - Google Maps URL'leri (…/@lat,lon,zoom… veya …q=lat,lon…)
    - Metin içinde geçen ilk lat,lon çifti
    """
    m = LATLON_RE.search(s)
    if not m:
        raise ValueError("Lat,lon bulunamadı. Örnek: 41.2397,41.9156 veya Google Maps linki verin.")
    lat, lon = float(m.group(1)), float(m.group(2))
    if not (-90 <= lat <= 90 and -180 <= lon <= 180):
        raise ValueError(f"Geçersiz aralık: {lat}, {lon}")
    return lat, lon

def handle_start(arg: str):
    lat, lon = parse_latlon(arg)
    run_point_report(lat, lon)

def repl():
    print("VerimGören komut modu. Örnek: /start 41.2397,41.9156  |  /start https://maps.google.com/...  |  /quit")
    while True:
        try:
            line = input("> ").strip()
        except (EOFError, KeyboardInterrupt):
            print()
            break
        if not line:
            continue
        if line.lower() in {"/q", "/quit", "/exit"}:
            break
        if line.lower().startswith("/start"):
            rest = line[6:].strip()
            if not rest:
                print("Kullanım: /start [lat lon] veya Google Maps linki")
                continue
            try:
                handle_start(rest)
            except Exception as e:
                print(f"Hata: {e}")
        else:
            print("Bilinmeyen komut. Sadece /start ve /quit desteklenir.")

def main():
    parser = argparse.ArgumentParser(description="VerimGören — /start lansmanı")
    parser.add_argument("query", nargs="*", help="'/start ...' veya direkt lat,lon / Google Maps linki")
    args = parser.parse_args()

    if not args.query:
        # Argüman yoksa REPL moduna gir
        repl()
        return

    text = " ".join(args.query).strip()
    # '/start ...' yazılmışsa kırp
    if text.lower().startswith("/start"):
        text = text[6:].strip()
        if not text:
            print("Kullanım: /start [lat lon] veya Google Maps linki")
            sys.exit(2)

    try:
        handle_start(text)
    except Exception as e:
        print(f"Hata: {e}")
        sys.exit(1)

# Notebook'ta argparse çakışmasın diye koruma:
if __name__ == "__main__" and "ipykernel" not in sys.modules:
    main()

# Notebook/REPL için kolay yardımcılar:
def start(text: str):
    """Notebook/REPL içinden: start('41.2397,41.9156') veya start('https://maps.google.com/...')"""
    handle_start(text)

def start_latlon(lat: float, lon: float):
    """Notebook/REPL içinden: start_latlon(41.2397, 41.9156)"""
    run_point_report(lat, lon)


In [11]:
start("40.67274314058153, 35.991192102260634")

📍 Konum: 40.67274, 35.99119

📌 Konum
- Enlem (latitude): 40.50 °
- Boylam (longitude): 36 °
- Enlem (LAT): 40.67 °
- Boylam (LON): 35.99 °

🌦️ İklim
- PAR (tümü) (ALLSKY_SFC_PAR_TOT_grp1): 0.81 MJ/m²/gün
- Kısa dalga (tümü) (ALLSKY_SFC_SW_DWN_grp1): 1.85 kWh/m²/gün
- Bulutluluk (CLOUD_AMT_grp1): 77.48 %
- Bulutluluk (gündüz) (CLOUD_AMT_DAY_grp1): 76.10 %
- Bulutluluk (gece) (CLOUD_AMT_NIGHT_grp1): 78.43 %
- Açık gün sayısı (CLRSKY_DAYS_grp1): 6 gün/ay
- Kısa dalga (açık gök) (CLRSKY_SFC_SW_DWN_grp1): 2.90 kWh/m²/gün
- Özgül nem (2 m) (QV2M_grp2): 3.36 g/kg
- Bağıl nem (2 m) (RH2M_grp2): 82.44 %
- Sıcaklık (2 m, ort.) (T2M_grp2): -0.35 °C
- Çiy noktası (T2MDEW_grp2): -3.32 °C
- Yaş termometre (T2MWET_grp2): -1.83 °C
- Maks. sıcaklık (T2M_MAX_grp2): 16.10 °C
- Min. sıcaklık (T2M_MIN_grp2): -19.53 °C
- Günlük sıcaklık aralığı (T2M_RANGE_grp2): 9.17 °C
- Kolon su buharı (TQV_grp2): 8.27 kg/m²
- Yüzey basıncı (PS_grp3): 91.61 kPa
- Denize indirgenmiş basınç (SLP_grp3): 101.88 kPa
- Rüzgar y

In [17]:
# -*- coding: utf-8 -*-
# VerimGören — Ürün Uygunluk Skoru (0–100), modüler
# Kaynak uyumu: FAO ECOCROP, FAO-56, FAO Water Quality Annex A1
# Not: Her modül 0–100 üretir. Girdisi olmayan modül skordan çıkarılır (ağırlıklar normalize edilir).

def _clip01(x):
    x = float(x)
    return 0.0 if x < 0.0 else (1.0 if x > 1.0 else x)

def _presence(val):
    return (val is not None) and (str(val).strip() != "")

def _mean_safe(vals):
    vals = [float(v) for v in vals if _presence(v)]
    return sum(vals)/len(vals) if vals else None

def _trapezoid_score(x, a, b, c, d):
    """
    a <= b <= c <= d; [a,b] artış, [b,c] plato (1), [c,d] azalış.
    Dönen skor: 0–100
    """
    if any(v is None for v in [x, a, b, c, d]):
        return None
    x = float(x); a, b, c, d = float(a), float(b), float(c), float(d)
    if a >= b or b > c or c >= d:
        return None  # parametre hatası
    if x <= a or x >= d:
        return 0.0
    if b <= x <= c:
        return 100.0
    if a < x < b:
        return 100.0 * (x - a) / (b - a)
    # c < x < d
    return 100.0 * (d - x) / (d - c)

def suitability_score(crop, env, weights=None, params=None):
    """
    crop: dict-like (örn: {'crop':'apple','tmin_abs':-20,'topt_min':16,'topt_max':24,'tmax_abs':35,
                           'pH_min':5.5,'pH_max':7.5,'ece_threshold_dSm':1.7,
                           'root_depth_m':2.0,'kc_initial':0.30,'kc_mid':0.85,'kc_end':0.50,
                           'texture_ok':'loam,sandy_loam','drainage_preference':'well-drained'})
    env : dict-like (NASA POWER + HWSD + arazi özet)
          Yaygın alan adları:
            T2M_grp2 (Tavg), T2M_MIN_grp2 (Tmin), T2M_MAX_grp2 (Tmax),
            RH2M_grp2, ALLSKY_SFC_SW_DWN_grp1 (kWh/m2/gün),
            PRECTOTCORR_grp4 (mm/gün),
            T_PH_H2O, T_ECE (dS/m), T_ESP (%), T_CACO3 (%), T_CEC_SOIL (cmol(+)/kg),
            AWC_MM_PER_M, T_USDA_TEX_DESC, DRAINAGE_DESC,
            ELEVATION_M, WS2M_MAX_grp3 (m/s), NIGHT_LIGHT
          (ET0 veya ETc sağlarsan su dengesi modülü de devreye girer.)
    weights: opsiyonel ağırlık sözlüğü (modül bazında override)
    params : opsiyonel eşik/ayar sözlüğü
    return: {'score': float|None, 'modules': {modul_adı: skor}, 'used_weights':{modul_adı: ağırlık}}
    """

    # --- Varsayılan ağırlıklar (toplam 100) ---
    W = {
        # İklim 40
        'thermal': 12, 'frost': 6, 'heat': 6, 'rad': 8, 'rh': 8,
        # Su dengesi 15
        'water': 15,
        # Toprak fizikokimya 25
        'ph': 8, 'ec': 8, 'soilphys': 5, 'taw': 4,
        # Kimyasal 10
        'esp': 3, 'caco3': 3, 'cec': 2,
        # Arazi/operasyonel 10
        'elev': 5, 'wind': 3, 'night': 2,
    }
    if isinstance(weights, dict):
        W.update(weights)

    # --- Parametreler (eşikler) ---
    P = {
        'rh_opt': 60.0, 'rh_span': 30.0,             # 60±30 → 0
        'rad_min': 1.2, 'rad_max': 2.8,              # kWh/m2/gün (genel)
        'frost_band': 5.0, 'heat_band': 5.0,         # ceza bant genişliği
        'esp_max_default': 8.0,                      # %
        'caco3_tol_default': 10.0,                   # %
        'cec_soft_thresholds': (8.0, 12.0),          # <8:40, 8-12:70, >=12:100
        'taw_ref_default': 120.0,                    # mm (ürün sınıfına göre ayarlanabilir)
        'texture_neighbors': {                       # doku yakınlığı
            'loam': {'sandy_loam','silt_loam','clay_loam'},
            'sandy_loam': {'loam'}, 'silt_loam': {'loam'}, 'clay_loam': {'loam'},
            'sandy_clay_loam': {'clay_loam','sandy_loam'},
            'silty_clay_loam': {'clay_loam','silt_loam'},
        },
        'drain_map': {                               # drenaj yorumlama
            'well': 100, 'moderately well': 70,
            'somewhat poorly': 40, 'poorly': 0, 'very poorly': 0
        }
    }
    if isinstance(params, dict):
        P.update(params)

    modules, usedW = {}, {}

    # ---- İklim modülleri ----
    Tavg = env.get('T2M_grp2')
    Tmin = env.get('T2M_MIN_grp2')
    Tmax = env.get('T2M_MAX_grp2')

    tmin_abs = crop.get('tmin_abs'); topt_min = crop.get('topt_min')
    topt_max = crop.get('topt_max'); tmax_abs = crop.get('tmax_abs')

    # 1) Termal (trapez)
    if all(_presence(v) for v in [Tavg, tmin_abs, topt_min, topt_max, tmax_abs]):
        modules['thermal'] = _trapezoid_score(Tavg, tmin_abs, topt_min, topt_max, tmax_abs)
        usedW['thermal'] = W['thermal']

    # 2) Don riski
    if all(_presence(v) for v in [Tmin, tmin_abs]):
        if float(Tmin) >= float(tmin_abs):
            modules['frost'] = 100.0
        else:
            dd = abs(float(tmin_abs) - float(Tmin))
            modules['frost'] = max(0.0, 100.0 - 100.0 * (dd / P['frost_band']))
        usedW['frost'] = W['frost']

    # 3) Sıcak stres
    if all(_presence(v) for v in [Tmax, tmax_abs]):
        if float(Tmax) <= float(tmax_abs):
            modules['heat'] = 100.0
        else:
            dd = abs(float(Tmax) - float(tmax_abs))
            modules['heat'] = max(0.0, 100.0 - 100.0 * (dd / P['heat_band']))
        usedW['heat'] = W['heat']

    # 4) Işınım
    R = env.get('ALLSKY_SFC_SW_DWN_grp1')  # kWh/m2/gün
    if _presence(R):
        rmin, rmax = float(P['rad_min']), float(P['rad_max'])
        modules['rad'] = 100.0 * _clip01((float(R) - rmin) / max(1e-6, (rmax - rmin)))
        usedW['rad'] = W['rad']

    # 5) Bağıl nem
    RH = env.get('RH2M_grp2')
    if _presence(RH):
        rh_opt = float(P['rh_opt']); span = float(P['rh_span'])
        modules['rh'] = 100.0 * _clip01(1.0 - ((float(RH) - rh_opt) / span) ** 2)
        usedW['rh'] = W['rh']

    # ---- Su dengesi (ETc - P) ----
    Pmm = env.get('PRECTOTCORR_grp4')  # mm/gün
    kc_avg = _mean_safe([crop.get('kc_initial'), crop.get('kc_mid'), crop.get('kc_end')])
    ETc = env.get('ETc')               # varsa doğrudan
    ET0 = env.get('ET0')               # varsa ETc = ET0 * kc_avg
    AWC = env.get('AWC_MM_PER_M')      # mm/m
    Zr  = crop.get('root_depth_m')     # m

    if _presence(Pmm) and _presence(kc_avg) and (_presence(ETc) or _presence(ET0)) and _presence(AWC) and _presence(Zr):
        if not _presence(ETc):
            ETc = float(ET0) * float(kc_avg)
        deficit = max(0.0, float(ETc) - float(Pmm))
        TAW = float(AWC) * float(Zr)  # mm
        denom = max(1.0, TAW / 15.0)  # 15 günlük tampon
        modules['water'] = 100.0 * _clip01(1.0 - deficit / denom)
        usedW['water'] = W['water']

    # ---- Toprak fizikokimya ----
    # 7) pH (trapez: omuzlar ±0.5)
    soil_pH = env.get('T_PH_H2O')
    pH_min = crop.get('pH_min'); pH_max = crop.get('pH_max')
    if all(_presence(v) for v in [soil_pH, pH_min, pH_max]):
        a = float(pH_min) - 0.5; b = float(pH_min); c = float(pH_max); d = float(pH_max) + 0.5
        modules['ph'] = _trapezoid_score(float(soil_pH), a, b, c, d)
        usedW['ph'] = W['ph']

    # 8) EC (tuzluluk)
    soil_EC = env.get('T_ECE')  # dS/m
    ec_thr  = crop.get('ece_threshold_dSm')
    if all(_presence(v) for v in [soil_EC, ec_thr]):
        thr = max(0.1, float(ec_thr))
        modules['ec'] = 100.0 * _clip01(1.0 - float(soil_EC)/thr)
        usedW['ec'] = W['ec']

    # 9) Doku & drenaj
    tex_ok = (crop.get('texture_ok') or "").lower().replace(" ", "")
    tex_ok_set = set([t.strip().lower() for t in tex_ok.split(",") if t.strip()])
    tex_env = (env.get('T_USDA_TEX_DESC') or "").strip().lower().replace(" ", "")
    drain_pref = (crop.get('drainage_preference') or "").strip().lower()
    drain_env  = (env.get('DRAINAGE_DESC') or "").strip().lower()

    # doku
    score_tex = None
    if tex_env:
        if tex_env in tex_ok_set:
            score_tex = 100.0
        else:
            neigh = {
                'loam': {'sandy_loam','silt_loam','clay_loam'},
                'sandy_loam': {'loam'}, 'silt_loam': {'loam'}, 'clay_loam': {'loam'},
                'sandy_clay_loam': {'clay_loam','sandy_loam'},
                'silty_clay_loam': {'clay_loam','silt_loam'},
            }
            score_tex = 60.0 if any((k in tex_ok_set and tex_env in neigh.get(k, set())) for k in tex_ok_set) else 0.0

    # drenaj
    score_drain = None
    if drain_pref and drain_env:
        def _normalize_drain(s):
            s = s.lower()
            if 'well' in s and 'moderate' not in s:
                return 'well'
            if 'moderately' in s:
                return 'moderately well'
            if 'very poorly' in s:
                return 'very poorly'
            if 'poorly' in s:
                return 'poorly'
            if 'somewhat' in s:
                return 'somewhat poorly'
            return None
        dkey = _normalize_drain(drain_env)
        dmap = {'well':100,'moderately well':70,'somewhat poorly':40,'poorly':0,'very poorly':0}
        score_drain = dmap.get(dkey, 70.0)

    if score_tex is not None or score_drain is not None:
        parts, wsum = [], 0.0
        if score_tex   is not None: parts.append((score_tex,   0.6)); wsum += 0.6
        if score_drain is not None: parts.append((score_drain, 0.4)); wsum += 0.4
        modules['soilphys'] = sum(s*w for s, w in parts) / (wsum if wsum else 1.0)
        usedW['soilphys'] = W['soilphys']

    # 10) TAW (AWC * kök)
    AWC = env.get('AWC_MM_PER_M'); Zr = crop.get('root_depth_m')
    if _presence(AWC) and _presence(Zr):
        TAW = float(AWC) * float(Zr)
        ref = float((params or {}).get('taw_ref_default', 120.0))
        modules['taw'] = 100.0 * _clip01(TAW / ref)
        usedW['taw'] = W['taw']

    # ---- Kimyasal / toksisite ----
    ESP = env.get('T_ESP') or env.get('ESP')
    if _presence(ESP):
        esp_max = float((params or {}).get('esp_max', 8.0))
        modules['esp'] = 100.0 * _clip01(1.0 - float(ESP)/max(1e-6, esp_max))
        usedW['esp'] = W['esp']

    CACO3 = env.get('T_CACO3') or env.get('S_CACO3') or env.get('CACO3')
    if _presence(CACO3):
        tol = float((params or {}).get('caco3_tol', 10.0))
        modules['caco3'] = 100.0 * _clip01(1.0 - float(CACO3)/max(1e-6, tol))
        usedW['caco3'] = W['caco3']

    CEC = env.get('T_CEC_SOIL')
    if _presence(CEC):
        lo, hi = (params or {}).get('cec_soft_thresholds', (8.0, 12.0))
        CEC = float(CEC)
        modules['cec'] = (40.0 if CEC < lo else (70.0 if CEC < hi else 100.0))
        usedW['cec'] = W['cec']

    # ---- Arazi / operasyonel ----
    elev = env.get('ELEVATION_M')
    elev_min = crop.get('elevation_min'); elev_max = crop.get('elevation_max')
    if _presence(elev):
        e = float(elev)
        if _presence(elev_min) and _presence(elev_max):
            a = float(elev_min) - 200.0; b = float(elev_min)
            c = float(elev_max); d = float(elev_max) + 200.0
            modules['elev'] = _trapezoid_score(e, a, b, c, d)
        else:
            if e < 1500:   modules['elev'] = 100.0
            elif e < 2000: modules['elev'] = 70.0
            elif e < 2500: modules['elev'] = 40.0
            else:          modules['elev'] = 0.0
        usedW['elev'] = W['elev']

    WSMAX = env.get('WS2M_MAX_grp3')
    if _presence(WSMAX):
        modules['wind'] = 100.0 * _clip01(1.0 - float(WSMAX)/15.0)  # ≥15 m/s → 0
        usedW['wind'] = W['wind']

    NL = env.get('NIGHT_LIGHT')
    if _presence(NL):
        modules['night'] = 100.0 * _clip01(float(NL)/5.0)           # 5 ve üzeri doygun
        usedW['night'] = W['night']

    # ---- Ağırlık normalize + toplam skor ----
    if not usedW:
        return {'score': None, 'modules': modules, 'used_weights': usedW}

    wsum = float(sum(usedW.values()))
    total = 0.0
    for k, sc in modules.items():
        if sc is None: 
            continue
        wk = usedW.get(k, 0.0) / wsum
        total += wk * float(sc)

    return {'score': round(total, 2), 'modules': {k: round(v,2) for k,v in modules.items()}, 'used_weights': usedW}


In [33]:
# -*- coding: utf-8 -*-
# VerimGören — Konumdan Özet + Doğru Yer Doğru Ürün (REPL, zengin çıktı)
#
# Nasıl kullanılır?
#   python verimgoren_app_v2.py
#   Konum girin (Google Maps linki YA DA 'lat,lon'):  40.67274,35.99119
#
# Notlar:
# - Bitki CSV (zorunlu): CROPS_CSV
# - İklim CSV (zorunlu): CSV_PATH (lat/long ve temel iklim sütunları)
# - Rakım/Gece ışığı rasterları opsiyonel: yoksa otomatik atlanır.
# - HWSD/toprak modülü opsiyonel: projende try_load_soil bulunursa kullanılır.
# - Hata toleranslı: eksik veri -> ilgili modül devre dışı.

import os, re, sys
from pathlib import Path
import pandas as pd
import numpy as np

# =========================
# 0) YOLLAR (gerekirse özelleştir)
# =========================
CROPS_CSV  = Path(os.environ.get("CROPS_CSV",  r"C:/Users/ataka/Desktop/MEHMET/VerimGören/notebooks/VerimGoren_Bitki_Parametreleri_Tam.csv"))
CSV_PATH   = Path(os.environ.get("CLIMATE_CSV",r"C:..\notebooks\data\climate\merged_climate_data.csv"))
ELEV_PATH  = Path(os.environ.get("ELEV_TIF",   r"C:..\data\processed\srtm_turkiye_cropped.tif"))      # opsiyonel
LIGHT_PATH = Path(os.environ.get("LIGHT_TIF",  r"C:..\data\processed\viirs_light_2024_turkey.tif"))   # opsiyonel

# =========================
# 1) Yardımcılar & yazım
# =========================
CATEGORY_ORDER = ["Konum", "İklim", "Arazi", "Gece Işığı", "Toprak", "Özet"]

def category_of(key: str) -> str:
    k = key.upper()
    if k in {"LAT", "LON", "LATITUDE", "LONGITUDE"}: return "Konum"
    if k in {"ELEVATION_M"}: return "Arazi"
    if k in {"NIGHT_LIGHT", "VIIRS_NTL"}: return "Gece Işığı"
    if (
        "_GRP" in k or k in {
            "T2M","T2M_MAX","T2M_MIN","T2M_RANGE","T2MDEW","T2MWET",
            "RH2M","QV2M","TQV","PS","SLP","WS2M","WS2M_MAX","WD2M",
            "PRECTOTCORR","TS","TO3","ALLSKY_SFC_SW_DWN","ALLSKY_SFC_PAR_TOT",
            "CLRSKY_SFC_SW_DWN","CLOUD_AMT","CLOUD_AMT_DAY","CLOUD_AMT_NIGHT","CLRSKY_DAYS","DISTANCE_KM"
        }
    ):
        return "İklim"
    if k in {
        "FAO90_DESC","T_USDA_TEX_DESC","S_USDA_TEX_DESC","T_TEXTURE_DESC",
        "T_SAND","T_SILT","T_CLAY","S_SAND","S_SILT","S_CLAY",
        "T_PH_H2O","S_PH_H2O","T_OC","S_OC","T_CEC_SOIL","S_CEC_SOIL",
        "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB",
        "T_CACO3","S_CACO3","AWC_MM_PER_M","DRAINAGE_DESC","T_ECE","S_ECE","T_ESP","S_ESP",
        "MU_GLOBAL"
    }:
        return "Toprak"
    return "Özet"

def format_value(v):
    if v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v))): return "-"
    try:
        f = float(v)
        if abs(f - round(f)) < 1e-9: return f"{int(round(f))}"
        return f"{f:.2f}"
    except Exception:
        return str(v)

# =========================
# 2) UYGUNLUK SKORU
# =========================
def _clip01(x): x=float(x); return 0.0 if x<0 else (1.0 if x>1 else x)
def _presence(val): return (val is not None) and (str(val).strip() != "")
def _mean_safe(vals):
    vals = [float(v) for v in vals if _presence(v)]
    return sum(vals)/len(vals) if vals else None

def _trapezoid_score(x, a, b, c, d):
    if any(v is None for v in [x,a,b,c,d]): return None
    x = float(x); a,b,c,d = float(a),float(b),float(c),float(d)
    if a >= b or b > c or c >= d: return None
    if x <= a or x >= d: return 0.0
    if b <= x <= c:      return 100.0
    if a < x < b:        return 100.0 * (x-a)/(b-a)
    return 100.0 * (d-x)/(d-c)

def suitability_score(crop, env, weights=None, params=None):
    # Ağırlıklar (toplam 100)
    W = {'thermal':12,'frost':6,'heat':6,'rad':8,'rh':8,'water':15,'ph':8,'ec':8,'soilphys':5,'taw':4,'esp':3,'caco3':3,'cec':2,'elev':5,'wind':3,'night':2}
    if isinstance(weights, dict): W.update(weights)
    P = {'rh_opt':60.0,'rh_span':30.0,'rad_min':1.2,'rad_max':2.8,'frost_band':5.0,'heat_band':5.0,'taw_ref_default':120.0}

    modules, usedW = {}, {}

    # İklim
    Tavg = env.get('T2M_grp2'); Tmin = env.get('T2M_MIN_grp2'); Tmax = env.get('T2M_MAX_grp2')
    tmin_abs = crop.get('tmin_abs'); topt_min = crop.get('topt_min'); topt_max = crop.get('topt_max'); tmax_abs = crop.get('tmax_abs')

    if all(_presence(v) for v in [Tavg,tmin_abs,topt_min,topt_max,tmax_abs]):
        modules['thermal'] = _trapezoid_score(Tavg, tmin_abs, topt_min, topt_max, tmax_abs); usedW['thermal']=W['thermal']

    if all(_presence(v) for v in [Tmin,tmin_abs]):
        if float(Tmin) >= float(tmin_abs): modules['frost']=100.0
        else:
            dd = abs(float(tmin_abs)-float(Tmin)); modules['frost']=max(0.0, 100.0-100.0*(dd/P['frost_band']))
        usedW['frost']=W['frost']

    if all(_presence(v) for v in [Tmax,tmax_abs]):
        if float(Tmax) <= float(tmax_abs): modules['heat']=100.0
        else:
            dd = abs(float(Tmax)-float(tmax_abs)); modules['heat']=max(0.0, 100.0-100.0*(dd/P['heat_band']))
        usedW['heat']=W['heat']

    R = env.get('ALLSKY_SFC_SW_DWN_grp1')
    if _presence(R):
        modules['rad'] = 100.0 * _clip01((float(R)-P['rad_min'])/max(1e-6,(P['rad_max']-P['rad_min'])))
        usedW['rad']=W['rad']

    RH = env.get('RH2M_grp2')
    if _presence(RH):
        modules['rh'] = 100.0 * _clip01(1.0 - ((float(RH)-P['rh_opt'])/P['rh_span'])**2)
        usedW['rh']=W['rh']

    # Su
    Pmm = env.get('PRECTOTCORR_grp4'); kc_avg = _mean_safe([crop.get('kc_initial'),crop.get('kc_mid'),crop.get('kc_end')])
    ETc = env.get('ETc'); ET0 = env.get('ET0'); AWC = env.get('AWC_MM_PER_M'); Zr = crop.get('root_depth_m')
    if _presence(Pmm) and _presence(kc_avg) and (_presence(ETc) or _presence(ET0)) and _presence(AWC) and _presence(Zr):
        if not _presence(ETc): ETc = float(ET0) * float(kc_avg)
        deficit = max(0.0, float(ETc) - float(Pmm))
        TAW = float(AWC) * float(Zr); denom = max(1.0, TAW/15.0)
        modules['water'] = 100.0 * _clip01(1.0 - deficit/denom); usedW['water']=W['water']

    # Toprak kimya/fizik
    soil_pH = env.get('T_PH_H2O'); pH_min = crop.get('pH_min'); pH_max = crop.get('pH_max')
    if all(_presence(v) for v in [soil_pH,pH_min,pH_max]):
        a=float(pH_min)-0.5; b=float(pH_min); c=float(pH_max); d=float(pH_max)+0.5
        modules['ph'] = _trapezoid_score(float(soil_pH), a,b,c,d); usedW['ph']=W['ph']

    soil_EC = env.get('T_ECE'); ec_thr = crop.get('ece_threshold_dSm') if 'ece_threshold_dSm' in crop else crop.get('ece_threshold_dsm')
    if all(_presence(v) for v in [soil_EC,ec_thr]):
        thr = max(0.1, float(ec_thr)); modules['ec'] = 100.0 * _clip01(1.0 - float(soil_EC)/thr); usedW['ec']=W['ec']

    # Doku & drenaj
    tex_ok = (crop.get('texture_ok') or "").lower().replace(" ","")
    tex_ok_set = set([t.strip().lower() for t in tex_ok.split(",") if t.strip()])
    tex_env = (env.get('T_USDA_TEX_DESC') or "").strip().lower().replace(" ","")
    drain_pref = (crop.get('drainage_preference') or "").strip().lower()
    drain_env  = (env.get('DRAINAGE_DESC') or "").strip().lower()

    score_tex = None
    if tex_env:
        if tex_env in tex_ok_set: score_tex = 100.0
        else:
            neigh = {
                'loam':{'sandy_loam','silt_loam','clay_loam'},
                'sandy_loam':{'loam'}, 'silt_loam':{'loam'}, 'clay_loam':{'loam'},
                'sandy_clay_loam':{'clay_loam','sandy_loam'},
                'silty_clay_loam':{'clay_loam','silt_loam'},
            }
            score_tex = 60.0 if any((k in tex_ok_set and tex_env in neigh.get(k,set())) for k in tex_ok_set) else 0.0

    def _norm_drain(s):
        s=s.lower()
        if 'well' in s and 'moderate' not in s: return 'well'
        if 'moderately' in s: return 'moderately well'
        if 'very poorly' in s: return 'very poorly'
        if 'poorly' in s: return 'poorly'
        if 'somewhat' in s: return 'somewhat poorly'
        return None

    score_drain = None
    if drain_pref and drain_env:
        dkey = _norm_drain(drain_env); dmap = {'well':100,'moderately well':70,'somewhat poorly':40,'poorly':0,'very poorly':0}
        score_drain = dmap.get(dkey, 70.0)

    if score_tex is not None or score_drain is not None:
        parts, wsum = [], 0.0
        if score_tex   is not None: parts.append((score_tex,   0.6)); wsum += 0.6
        if score_drain is not None: parts.append((score_drain, 0.4)); wsum += 0.4
        modules['soilphys'] = sum(s*w for s,w in parts) / (wsum if wsum else 1.0); usedW['soilphys']=W['soilphys']

    if _presence(AWC) and _presence(Zr):
        TAW = float(AWC)*float(Zr); ref = float((params or {}).get('taw_ref_default',120.0))
        modules['taw'] = 100.0 * _clip01(TAW/ref); usedW['taw']=W['taw']

    ESP = env.get('T_ESP') or env.get('ESP')
    if _presence(ESP):
        modules['esp'] = 100.0 * _clip01(1.0 - float(ESP)/8.0); usedW['esp']=W['esp']

    CACO3 = env.get('T_CACO3') or env.get('S_CACO3') or env.get('CACO3')
    if _presence(CACO3):
        modules['caco3'] = 100.0 * _clip01(1.0 - float(CACO3)/10.0); usedW['caco3']=W['caco3']

    CEC = env.get('T_CEC_SOIL')
    if _presence(CEC):
        CEC = float(CEC); modules['cec'] = 40.0 if CEC<8.0 else (70.0 if CEC<12.0 else 100.0); usedW['cec']=W['cec']

    elev = env.get('ELEVATION_M'); elev_min=crop.get('elevation_min'); elev_max=crop.get('elevation_max')
    if _presence(elev):
        e=float(elev)
        if _presence(elev_min) and _presence(elev_max):
            a=float(elev_min)-200.0; b=float(elev_min); c=float(elev_max); d=float(elev_max)+200.0
            modules['elev'] = _trapezoid_score(e,a,b,c,d)
        else:
            modules['elev'] = 100.0 if e<1500 else (70.0 if e<2000 else (40.0 if e<2500 else 0.0))
        usedW['elev']=W['elev']

    WSMAX = env.get('WS2M_MAX_grp3')
    if _presence(WSMAX):
        modules['wind'] = 100.0 * _clip01(1.0 - float(WSMAX)/15.0); usedW['wind']=W['wind']

    NL = env.get('NIGHT_LIGHT')
    if _presence(NL):
        modules['night'] = 100.0 * _clip01(float(NL)/5.0); usedW['night']=W['night']

    if not usedW: 
        return {'score': None, 'modules': modules, 'used_weights': usedW}
    wsum = float(sum(usedW.values())); total = 0.0
    for k, sc in modules.items():
        if sc is None: continue
        wk = usedW.get(k,0.0)/wsum; total += wk * float(sc)
    return {'score': round(total,2), 'modules': {k:round(v,2) for k,v in modules.items()}, 'used_weights': usedW}

# =========================
# 3) ENV VERİ TOPLAMA
# =========================
try:
    import rasterio
except Exception:
    rasterio = None

def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def load_climate_nearest(csv_path: Path, lat: float, lon: float):
    if not csv_path.exists():
        raise FileNotFoundError(f"İklim CSV bulunamadı: {csv_path}")
    df = pd.read_csv(csv_path)
    if not {"latitude","longitude"}.issubset(df.columns):
        raise ValueError("İklim CSV'de 'latitude' ve 'longitude' sütunları yok.")
    dist = haversine(lat, lon, df["latitude"].values, df["longitude"].values)
    i = int(np.argmin(dist))
    row = df.iloc[i].to_dict(); row["DISTANCE_KM"] = float(dist[i])
    return row

def sample_raster(path: Path, lon: float, lat: float):
    try:
        if rasterio is None or not Path(path).exists(): return None
        with rasterio.open(path) as ds:
            r, c = ds.index(lon, lat)
            val = ds.read(1)[r, c]
            if ds.nodata is not None and val == ds.nodata: return None
            return float(val)
    except Exception:
        return None

# Projeden HWSD try_load_soil arar (opsiyonel)
def _load_project_try_load_soil():
    cand = ["verimgoren_helpers", "helpers", "hwsd_helpers"]
    for name in cand:
        try:
            mod = __import__(name, fromlist=['try_load_soil'])
            if hasattr(mod, "try_load_soil"):
                return getattr(mod, "try_load_soil")
        except Exception:
            continue
    return None
TRY_LOAD_SOIL_FN = _load_project_try_load_soil()

def try_load_soil(lat: float, lon: float):
    if callable(TRY_LOAD_SOIL_FN):
        try:
            return TRY_LOAD_SOIL_FN(lat, lon)
        except Exception:
            return None
    return None

# =========================
# 4) GİRİŞ PARSE (sağlam)
# =========================
_NUM_RE = r'[-+]?\d+(?:\.\d+)?'
def parse_latlon(text: str):
    """
    Örnekler:
      '41.2397,41.9156' | '41.2397 41.9156'
      '...maps.google.com/?q=41.2397,41.9156'
      '...maps.google.com/@41.2397,41.9156,12z'
      '📍 Konum: 41.2397, 41.9156'
      'lat=41.2397 lon=41.9156'
      Metindeki ilk iki sayı
    """
    if text is None: raise ValueError("Boş giriş verildi.")
    s = str(text)

    m = re.search(r'[?&]q=(' + _NUM_RE + ')[, ]+(' + _NUM_RE + ')', s)
    if not m: m = re.search(r'@(' + _NUM_RE + ')[, ]+(' + _NUM_RE + ')', s)
    if not m: m = re.search(r'lat[^-+0-9]*(' + _NUM_RE + ').*?lon[^-+0-9]*(' + _NUM_RE + ')', s, re.IGNORECASE | re.DOTALL)

    if not m:
        nums = re.findall(_NUM_RE, s)
        if len(nums) >= 2: lat, lon = float(nums[0]), float(nums[1])
        else: raise ValueError("Lat,lon bulunamadı. Örnek: 41.2397,41.9156 veya Google Maps linki verin.")
    else:
        lat, lon = float(m.group(1)), float(m.group(2))

    if not (-90 <= lat <= 90 and -180 <= lon <= 180):
        raise ValueError(f"Geçersiz aralık: {lat}, {lon}")
    return lat, lon

# =========================
# 5) ENV oluşturma + özet yazdırma
# =========================
def _pick(d, *keys):
    for k in keys:
        if k in d: return d[k]
    return None

def build_env(lat: float, lon: float) -> dict:
    env = {}
    clim = load_climate_nearest(CSV_PATH, lat, lon)

    # İklim başlıkları (ad varyasyonlarına toleranslı)
    env['latitude']              = _pick(clim,'latitude');  env['longitude'] = _pick(clim,'longitude')
    env['T2M_grp2']              = _pick(clim,'T2M_grp2','T2M_GRP2','T2M')
    env['T2M_MIN_grp2']          = _pick(clim,'T2M_MIN_grp2','T2M_MIN_GRP2','T2M_MIN')
    env['T2M_MAX_grp2']          = _pick(clim,'T2M_MAX_grp2','T2M_MAX_GRP2','T2M_MAX')
    env['RH2M_grp2']             = _pick(clim,'RH2M_grp2','RH2M_GRP2','RH2M')
    env['ALLSKY_SFC_SW_DWN_grp1']= _pick(clim,'ALLSKY_SFC_SW_DWN_grp1','ALLSKY_SFC_SW_DWN_GRP1','ALLSKY_SFC_SW_DWN')
    env['PRECTOTCORR_grp4']      = _pick(clim,'PRECTOTCORR_grp4','PRECTOTCORR_GRP4','PRECTOTCORR')
    env['WS2M_MAX_grp3']         = _pick(clim,'WS2M_MAX_grp3','WS2M_MAX_GRP3','WS2M_MAX')
    env['DISTANCE_KM']           = clim.get('DISTANCE_KM')

    # Opsiyonel rasterlar
    elev  = sample_raster(ELEV_PATH, lon, lat)
    night = sample_raster(LIGHT_PATH, lon, lat)
    if elev  is not None: env['ELEVATION_M'] = elev
    if night is not None: env['NIGHT_LIGHT'] = night

    # Opsiyonel HWSD/toprak
    soil = try_load_soil(lat, lon)
    if soil:
        def pick_soil(*keys):
            for k in keys:
                if k in soil and soil[k] is not None: return soil[k]
            return None
        env['MU_GLOBAL']       = pick_soil('MU_GLOBAL')
        env['T_USDA_TEX_DESC'] = pick_soil('T_USDA_TEX_DESC')
        env['DRAINAGE_DESC']   = pick_soil('DRAINAGE_DESC')
        env['AWC_MM_PER_M']    = pick_soil('AWC_MM_PER_M')
        env['T_PH_H2O']        = pick_soil('T_PH_H2O')
        env['T_ECE']           = pick_soil('T_ECE')
        env['T_ESP']           = pick_soil('T_ESP')
        env['T_CACO3']         = pick_soil('T_CACO3')
        env['T_CEC_SOIL']      = pick_soil('T_CEC_SOIL')

    return env

def print_env_summary(env: dict):
    print("\n✅ Veriler çekiliyor ve özet hazırlanıyor...\n")
    # Zengin, okunabilir özet
    # Konum
    lat = env.get('latitude'); lon = env.get('longitude')
    if _presence(lat) and _presence(lon):
        print("📌 Konum")
        print(f"- Enlem (lat): {format_value(lat)}")
        print(f"- Boylam (lon): {format_value(lon)}\n")

    # İklim
    print("🌦️ İklim")
    klim_keys = [
        ('ALLSKY_SFC_SW_DWN (kWh/m²/gün)','ALLSKY_SFC_SW_DWN_grp1'),
        ('T2M (°C)','T2M_grp2'),
        ('T2M_MIN (°C)','T2M_MIN_grp2'),
        ('T2M_MAX (°C)','T2M_MAX_grp2'),
        ('RH2M (%)','RH2M_grp2'),
        ('Yağış PRECTOTCORR (mm/gün)','PRECTOTCORR_grp4'),
        ('Rüzgar max WS2M_MAX (m/s)','WS2M_MAX_grp3'),
    ]
    for label, k in klim_keys:
        if k in env: print(f"- {label}: {format_value(env[k])}")
    if 'DISTANCE_KM' in env:
        print(f"- En yakın iklim hücresi uzaklığı (km): {format_value(env['DISTANCE_KM'])}")
        try:
            d = float(env['DISTANCE_KM'])
            if d > 50:
                print("  ⚠️ Bu nokta için iklim verisi uzak gridden geliyor; sonuçlar belirsizlik içerir.")
        except Exception:
            pass
    print()

    # Arazi / Gece ışığı
    if 'ELEVATION_M' in env:
        print("⛰️ Arazi")
        print(f"- Rakım (m): {format_value(env['ELEVATION_M'])}\n")
    if 'NIGHT_LIGHT' in env:
        print("🌃 Gece Işığı")
        print(f"- VIIRS NTL (bağıl): {format_value(env['NIGHT_LIGHT'])}\n")

    # Toprak
    top_keys = [
        ('USDA doku (üst)','T_USDA_TEX_DESC'),
        ('Drenaj','DRAINAGE_DESC'),
        ('AWC (mm/m)','AWC_MM_PER_M'),
        ('pH (üst)','T_PH_H2O'),
        ('EC (dS/m, üst)','T_ECE'),
        ('ESP (%)','T_ESP'),
        ('CaCO3 (%)','T_CACO3'),
        ('CEC (cmol(+)/kg)','T_CEC_SOIL'),
        ('MU_GLOBAL','MU_GLOBAL'),
    ]
    has_soil = any(k in env for _,k in top_keys)
    if has_soil:
        print("🌱 Toprak")
        for label, k in top_keys:
            if k in env: print(f"- {label}: {format_value(env[k])}")
        print()
    else:
        print("🌱 Toprak\n- (HWSD/toprak bilgisi bulunamadı — opsiyoneldir)\n")

# =========================
# 6) Bitki CSV & sıralama (modül analizi ile)
# =========================
def load_crops(crops_csv_path: Path) -> pd.DataFrame:
    if not crops_csv_path.exists():
        raise FileNotFoundError(f"Bitki CSV bulunamadı: {crops_csv_path}")
    df = pd.read_csv(crops_csv_path)
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def row_to_crop_dict(row: pd.Series) -> dict:
    d = row.to_dict()
    if 'ece_threshold_dsm' in d and 'ece_threshold_dSm' not in d:
        d['ece_threshold_dSm'] = d['ece_threshold_dsm']
    if not str(d.get('texture_ok','')).strip():
        d['texture_ok'] = ''
    return d

def weakest_modules(mod_dict, n=2):
    # En zayıf n modülü (skoru en düşük olanlar)
    if not mod_dict: return []
    items = [(k, v) for k,v in mod_dict.items() if v is not None]
    if not items: return []
    items.sort(key=lambda x: x[1])  # düşükten yükseğe
    return [f"{k}:{v:.0f}" for k,v in items[:n]]

def score_and_rank(env: dict, crops_df: pd.DataFrame, top_k: int = 10):
    results = []
    for _, row in crops_df.iterrows():
        crop = row_to_crop_dict(row)
        res = suitability_score(crop, env)
        score = res.get('score')
        if score is None:
            continue
        results.append({
            'crop': crop.get('crop'),
            'common_name_tr': crop.get('common_name_tr'),
            'score': score,
            'modules': res.get('modules', {})
        })
    if not results:
        print("⚠️ Skor üretilemedi (gerekli çevre/bitki alanları eksik olabilir).")
        return

    results = sorted(results, key=lambda x: x['score'], reverse=True)
    top = results[:max(1, top_k)]

    print("— Ürün Uygunluk Skoru (0–100) —")
    # Tablo başlığı
    print(f"{'Ürün':<22}  {'Skor':>5}  {'Neyi sınırlıyor? (en zayıf 2 modül)':<40}")
    print("-"*72)
    for r in top:
        trname = (r['common_name_tr'] or r['crop'])
        wmods = ", ".join(weakest_modules(r['modules'], n=2)) or "-"
        print(f"{trname:<22}  {r['score']:>5.1f}  {wmods:<40}")

    first3 = [ (r['common_name_tr'] or r['crop']) for r in top[:3] ]
    print("\nÖnerilen ilk 3: " + ", ".join(first3))
    print(f"\n🏆 En uygun ürün: {first3[0]}")

# =========================
# 7) Girdi isteme (REPL gibi ama tek seferde)
# =========================
def ask_for_location():
    try:
        line = input("Konum girin (Google Maps linki YA DA 'lat,lon'):  ").strip()
        if not line:
            raise ValueError("Boş giriş.")
        return line
    except (EOFError, KeyboardInterrupt):
        print("\nİptal edildi.")
        sys.exit(1)

def main_once():
    # 1) Girdi
    query = ask_for_location()
    try:
        lat, lon = parse_latlon(query)
    except Exception as e:
        print(f"⚠️ Konum çözümlenemedi: {e}")
        sys.exit(2)

    print(f"\n📍 Konum: {lat:.5f}, {lon:.5f}")
    print("ℹ️ İstekte bulunduğunuz konum alındı. Veriler çekiliyor...\n")

    # 2) ENV
    try:
        env = build_env(lat, lon)
    except FileNotFoundError as e:
        print(f"❌ {e}\n👉 Yol(ları) kontrol edin ya da çevre değişkeniyle ayarlayın.\n"
              f"   CROPS_CSV={CROPS_CSV}\n   CLIMATE_CSV={CSV_PATH}\n"
              f"   (Rakım/Gece ışığı opsiyoneldir: ELEV_TIF, LIGHT_TIF)")
        sys.exit(1)
    except Exception as e:
        print(f"⚠️ ENV oluştururken hata: {type(e).__name__}: {e}")
        sys.exit(1)

    # 3) Özet
    print_env_summary(env)

    # 4) Skor & sıralama
    try:
        crops_df = load_crops(CROPS_CSV)
        print("🔎 Uygunluk hesaplanıyor ve sıralanıyor...\n")
        score_and_rank(env, crops_df, top_k=10)
        print("\n— bitti —\n")
    except FileNotFoundError as e:
        print(f"❌ {e}\n👉 Bitki CSV yolunu kontrol edin: {CROPS_CSV}")
        sys.exit(1)
    except Exception as e:
        print(f"⚠️ Skorlama sırasında hata: {type(e).__name__}: {e}")
        sys.exit(1)

if __name__ == "__main__":
    # Parametre verilirse önce onu dener; verilmezse kullanıcıdan ister.
    if len(sys.argv) > 1:
        query = " ".join(sys.argv[1:])
        try:
            lat, lon = parse_latlon(query)
        except Exception as e:
            print(f"⚠️ Konum çözümlenemedi: {e}")
            sys.exit(2)
        print(f"\n📍 Konum: {lat:.5f}, {lon:.5f}")
        print("ℹ️ İstekte bulunduğunuz konum alındı. Veriler çekiliyor...\n")
        try:
            env = build_env(lat, lon)
            print_env_summary(env)
            crops_df = load_crops(CROPS_CSV)
            print("🔎 Uygunluk hesaplanıyor ve sıralanıyor...\n")
            score_and_rank(env, crops_df, top_k=10)
            print("\n— bitti —\n")
        except Exception as e:
            print(f"⚠️ Çalışma sırasında hata: {type(e).__name__}: {e}")
            sys.exit(1)
    else:
        main_once()



📍 Konum: -9.00000, 2.00000
ℹ️ İstekte bulunduğunuz konum alındı. Veriler çekiliyor...


✅ Veriler çekiliyor ve özet hazırlanıyor...

📌 Konum
- Enlem (lat): 37
- Boylam (lon): 27.50

🌦️ İklim
- ALLSKY_SFC_SW_DWN (kWh/m²/gün): 2.31
- T2M (°C): 12.32
- T2M_MIN (°C): 0.76
- T2M_MAX (°C): 19.14
- RH2M (%): 74.34
- Yağış PRECTOTCORR (mm/gün): 4.91
- Rüzgar max WS2M_MAX (m/s): 16.71
- En yakın iklim hücresi uzaklığı (km): 5764.69
  ⚠️ Bu nokta için iklim verisi uzak gridden geliyor; sonuçlar belirsizlik içerir.

🌱 Toprak
- (HWSD/toprak bilgisi bulunamadı — opsiyoneldir)

🔎 Uygunluk hesaplanıyor ve sıralanıyor...

— Ürün Uygunluk Skoru (0–100) —
Ürün                     Skor  Neyi sınırlıyor? (en zayıf 2 modül)     
------------------------------------------------------------------------
fasulye (kuru)           83.1  wind:0, rad:69                          
kanola/kolza             83.1  wind:0, rad:69                          
nohut                    83.1  wind:0, rad:69                   

In [35]:
# -*- coding: utf-8 -*-
# VerimGören — Konumdan: Tüm Veriler (UI) + Ürün Uygunluk Skoru (0–100)
# Kullanım:
#   python verimgoren_run.py
#   Konum girin (Google Maps linki YA DA 'lat,lon'):  40.67274,35.99119

import os, re, sys
from pathlib import Path
import pandas as pd
import numpy as np

# =========================
# 0) YOLLAR — KENDİ SİSTEMİNE GÖRE GÜNCELLE
# =========================
CROPS_CSV  = Path(r"C:/Users/ataka/Desktop/MEHMET/VerimGören/notebooks/VerimGoren_Bitki_Parametreleri_Tam.csv")
CSV_PATH   = Path(r"C:..\notebooks\data\climate\merged_climate_data.csv")  # İklim (0.5° grid)
ELEV_PATH  = Path(r"C:..\data\processed\srtm_turkiye_cropped.tif")         # Opsiyonel
LIGHT_PATH = Path(r"C:..\data\processed\viirs_light_2024_turkey.tif")      # Opsiyonel

# HWSD (Toprak)
HWSD_MDB = Path(r"C:/Users/ataka/Desktop/MEHMET/VerimGören/notebooks/hwsd_data/HWSD.mdb")
HWSD_RAS = Path(r"C:/Users/ataka/Desktop/MEHMET/VerimGören/notebooks/hwsd_data/hwsd.bil")

# =========================
# 1) BAŞLIK-ANLAM-BİRİM SÖZLÜĞÜ + KATEGORİ
# =========================
CATEGORY_ORDER = ["Konum", "İklim", "Arazi", "Gece Işığı", "Toprak", "Özet"]

def category_of(key: str) -> str:
    k = key.upper()
    if k in {"LAT", "LON", "LATITUDE", "LONGITUDE"}: return "Konum"
    if k in {"ELEVATION_M"}: return "Arazi"
    if k in {"NIGHT_LIGHT", "VIIRS_NTL"}: return "Gece Işığı"
    if ("_GRP" in k) or k in {
        "T2M","T2M_MAX","T2M_MIN","T2M_RANGE","T2MDEW","T2MWET","RH2M","QV2M","TQV","PS","SLP",
        "WS2M","WS2M_MAX","WD2M","PRECTOTCORR","TS","TO3","ALLSKY_SFC_SW_DWN","ALLSKY_SFC_PAR_TOT",
        "CLRSKY_SFC_SW_DWN","CLOUD_AMT","CLOUD_AMT_DAY","CLOUD_AMT_NIGHT","CLRSKY_DAYS","DISTANCE_KM"
    }:
        return "İklim"
    if k in {
        "FAO90_DESC","T_USDA_TEX_DESC","S_USDA_TEX_DESC","T_TEXTURE_DESC",
        "T_SAND","T_SILT","T_CLAY","S_SAND","S_SILT","S_CLAY",
        "T_PH_H2O","S_PH_H2O","T_OC","S_OC","T_CEC_SOIL","S_CEC_SOIL",
        "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB",
        "T_CACO3","S_CACO3","AWC_MM_PER_M","DRAINAGE_DESC","T_ECE","S_ECE","T_ESP","S_ESP",
        "MU_GLOBAL"
    }:
        return "Toprak"
    if k in {"DISTANCE_KM_IDW"}: return "Özet"
    return "Özet"

VAR_META = {
    # Konum
    "latitude":  {"title_tr":"Enlem","unit":"°"},
    "longitude": {"title_tr":"Boylam","unit":"°"}, "LAT":{"title_tr":"Enlem","unit":"°"}, "LON":{"title_tr":"Boylam","unit":"°"},
    # İklim (özet; diğerleri anahtardan türetilecek)
    "ALLSKY_SFC_PAR_TOT": {"title_tr":"PAR (tümü)","unit":"MJ/m²/gün"},
    "ALLSKY_SFC_SW_DWN":  {"title_tr":"Kısa dalga (tümü)","unit":"kWh/m²/gün"},
    "CLRSKY_SFC_SW_DWN":  {"title_tr":"Kısa dalga (açık gök)","unit":"kWh/m²/gün"},
    "CLRSKY_DAYS":        {"title_tr":"Açık gün sayısı","unit":"gün/ay"},
    "CLOUD_AMT":          {"title_tr":"Bulutluluk","unit":"%"},
    "CLOUD_AMT_DAY":      {"title_tr":"Bulutluluk (gündüz)","unit":"%"},
    "CLOUD_AMT_NIGHT":    {"title_tr":"Bulutluluk (gece)","unit":"%"},
    "QV2M":               {"title_tr":"Özgül nem (2 m)","unit":"g/kg"},
    "RH2M":               {"title_tr":"Bağıl nem (2 m)","unit":"%"},
    "T2M":                {"title_tr":"Sıcaklık (2 m, ort.)","unit":"°C"},
    "T2M_MAX":            {"title_tr":"Maks. sıcaklık","unit":"°C"},
    "T2M_MIN":            {"title_tr":"Min. sıcaklık","unit":"°C"},
    "T2M_RANGE":          {"title_tr":"Günlük sıcaklık aralığı","unit":"°C"},
    "T2MDEW":             {"title_tr":"Çiy noktası","unit":"°C"},
    "T2MWET":             {"title_tr":"Yaş termometre","unit":"°C"},
    "TQV":                {"title_tr":"Kolon su buharı","unit":"kg/m²"},
    "PS":                 {"title_tr":"Yüzey basıncı","unit":"kPa"},
    "SLP":                {"title_tr":"Denize indirgenmiş basınç","unit":"kPa"},
    "WD2M":               {"title_tr":"Rüzgar yönü (2 m)","unit":"°"},
    "WS2M":               {"title_tr":"Rüzgar hızı (2 m)","unit":"m/s"},
    "WS2M_MAX":           {"title_tr":"Maks. rüzgar (2 m)","unit":"m/s"},
    "PRECTOTCORR":        {"title_tr":"Toplam yağış (düz.)","unit":"mm/gün"},
    "TO3":                {"title_tr":"Toplam ozon","unit":"DU"},
    "TS":                 {"title_tr":"Yüzey sıcaklığı","unit":"°C"},
    "DISTANCE_KM":        {"title_tr":"Uzaklık (iklim pikseli)","unit":"km"},
    # Arazi & Gece ışığı
    "ELEVATION_M": {"title_tr":"Rakım","unit":"m"},
    "NIGHT_LIGHT": {"title_tr":"Gece ışığı","unit":"-"},
    # Toprak
    "FAO90_DESC":{"title_tr":"FAO-90 sınıfı","unit":"-"},
    "T_USDA_TEX_DESC":{"title_tr":"USDA doku (üst)","unit":"-"},
    "S_USDA_TEX_DESC":{"title_tr":"USDA doku (alt)","unit":"-"},
    "T_TEXTURE_DESC":{"title_tr":"Üst doku (coarse/medium/fine)","unit":"-"},
    "T_SAND":{"title_tr":"Kum (üst)","unit":"%"},
    "T_SILT":{"title_tr":"Silt (üst)","unit":"%"},
    "T_CLAY":{"title_tr":"Kil (üst)","unit":"%"},
    "S_SAND":{"title_tr":"Kum (alt)","unit":"%"},
    "S_SILT":{"title_tr":"Silt (alt)","unit":"%"},
    "S_CLAY":{"title_tr":"Kil (alt)","unit":"%"},
    "T_PH_H2O":{"title_tr":"pH (üst)","unit":"-"},
    "S_PH_H2O":{"title_tr":"pH (alt)","unit":"-"},
    "T_OC":{"title_tr":"Organik C (üst)","unit":"%"},
    "S_OC":{"title_tr":"Organik C (alt)","unit":"%"},
    "T_CEC_SOIL":{"title_tr":"CEC (üst)","unit":"cmol(+)/kg"},
    "S_CEC_SOIL":{"title_tr":"CEC (alt)","unit":"cmol(+)/kg"},
    "T_CEC_CLAY":{"title_tr":"CEC (kil, üst)","unit":"cmol(+)/kg"},
    "S_CEC_CLAY":{"title_tr":"CEC (kil, alt)","unit":"cmol(+)/kg"},
    "T_BS":{"title_tr":"Baz doygunluğu (üst)","unit":"%"},
    "S_BS":{"title_tr":"Baz doygunluğu (alt)","unit":"%"},
    "T_TEB":{"title_tr":"Toplam değişebilir baz (üst)","unit":"cmol(+)/kg"},
    "S_TEB":{"title_tr":"Toplam değişebilir baz (alt)","unit":"cmol(+)/kg"},
    "T_CACO3":{"title_tr":"Kireç CaCO₃ (üst)","unit":"%"},
    "S_CACO3":{"title_tr":"Kireç CaCO₃ (alt)","unit":"%"},
    "T_ECE":{"title_tr":"EC (üst)","unit":"dS/m"},
    "S_ECE":{"title_tr":"EC (alt)","unit":"dS/m"},
    "T_ESP":{"title_tr":"ESP (üst)","unit":"%"},
    "S_ESP":{"title_tr":"ESP (alt)","unit":"%"},
    "AWC_MM_PER_M":{"title_tr":"Kullanılabilir su (AWC)","unit":"mm/m"},
    "DRAINAGE_DESC":{"title_tr":"Drenaj","unit":"-"},
    "MU_GLOBAL":{"title_tr":"Harita birimi (MU)","unit":"-"},
}

def meta_of(key: str):
    m = VAR_META.get(key) or VAR_META.get(key.upper())
    if m: return m["title_tr"], m.get("unit","-")
    # otomatik başlık
    base = key.replace("_grp1","").replace("_grp2","").replace("_grp3","").replace("_grp4","")
    return base, "-"

def format_value(v):
    if v is None or (isinstance(v, float) and (np.isnan(v) or np.isinf(v))): return "-"
    try:
        f = float(v)
        if abs(f - round(f)) < 1e-9: return f"{int(round(f))}"
        return f"{f:.2f}"
    except Exception:
        return str(v)

# =========================
# 2) HWSD (TOPRAK) BAĞLAYICI
# =========================
try:
    import rasterio, pyodbc
except Exception:
    rasterio = None; pyodbc = None

def _read_table_access(table_name, mdb_path=HWSD_MDB) -> pd.DataFrame:
    if pyodbc is None:
        raise RuntimeError("pyodbc yüklü değil; HWSD.mdb okunamadı.")
    mdb_path = Path(mdb_path).resolve()
    if not mdb_path.exists():
        raise FileNotFoundError(f"HWSD.mdb bulunamadı: {mdb_path}")
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={str(mdb_path).replace('\\','/')};"
    cn = pyodbc.connect(conn_str)
    try:
        return pd.read_sql(f"SELECT * FROM {table_name}", cn)
    finally:
        cn.close()

def _normalize_cols(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out.columns = [c.strip().upper() for c in df.columns]
    return out

def _mu_from_latlon(lat: float, lon: float, raster_path=HWSD_RAS) -> int:
    if rasterio is None:
        raise RuntimeError("rasterio yok; hwsd.bil okunamadı.")
    rp = Path(raster_path).resolve()
    with rasterio.open(rp) as src:
        r, c = src.index(lon, lat)
        mu = int(src.read(1)[r, c])
    if mu <= 0:
        raise ValueError(f"Geçersiz MU_GLOBAL={mu}")
    return mu

_HWSD_DOM = None
def _load_hwsd_dom():
    global _HWSD_DOM
    if _HWSD_DOM is not None:
        return _HWSD_DOM
    hwsd = _normalize_cols(_read_table_access("HWSD_DATA"))

    def _safe(name):
        try: return _normalize_cols(_read_table_access(name))
        except Exception: return None

    tex   = _safe("D_TEXTURE")
    utex  = _safe("D_USDA_TEX_CLASS")
    awc   = _safe("D_AWC")
    drn   = _safe("D_DRAINAGE")
    sym90 = _safe("D_SYMBOL90")

    def _lut(df, out_code, out_desc):
        if isinstance(df, pd.DataFrame):
            cols = set(df.columns)
            code = "CODE" if "CODE" in cols else None
            desc = "DESCRIPTION" if "DESCRIPTION" in cols else ("VALUE" if "VALUE" in cols else None)
            if code and desc:
                return df.rename(columns={code:out_code, desc:out_desc})[[out_code, out_desc]]
        return None

    tex_lut   = _lut(tex,  "T_TEXTURE", "T_TEXTURE_DESC")
    usda_lutT = _lut(utex, "T_USDA_TEX_CLASS", "T_USDA_TEX_DESC")
    usda_lutS = _lut(utex, "S_USDA_TEX_CLASS", "S_USDA_TEX_DESC")
    awc_lut   = _lut(awc,  "AWC_CLASS", "AWC_MM_PER_M")
    drn_lut   = _lut(drn,  "DRAINAGE", "DRAINAGE_DESC")
    sym90_lut = _lut(sym90,"SU_CODE90", "FAO90_DESC")

    df = hwsd.copy()
    if tex_lut   is not None and "T_TEXTURE"        in df: df = df.merge(tex_lut,   on="T_TEXTURE",         how="left")
    if usda_lutT is not None and "T_USDA_TEX_CLASS" in df: df = df.merge(usda_lutT, on="T_USDA_TEX_CLASS",  how="left")
    if usda_lutS is not None and "S_USDA_TEX_CLASS" in df: df = df.merge(usda_lutS, on="S_USDA_TEX_CLASS",  how="left")
    if awc_lut   is not None and "AWC_CLASS"        in df: df = df.merge(awc_lut,   on="AWC_CLASS",         how="left")
    if drn_lut   is not None and "DRAINAGE"         in df: df = df.merge(drn_lut,   on="DRAINAGE",          how="left")
    if sym90_lut is not None and "SU_CODE90"        in df: df = df.merge(sym90_lut, on="SU_CODE90",         how="left")

    num_cols = [
        "AWC_MM_PER_M","T_PH_H2O","S_PH_H2O","T_OC","S_OC",
        "T_CLAY","T_SILT","T_SAND","S_CLAY","S_SILT","S_SAND",
        "T_ECE","S_ECE","T_ESP","S_ESP","T_CEC_SOIL","S_CEC_SOIL",
        "T_CEC_CLAY","S_CEC_CLAY","T_BS","S_BS","T_TEB","S_TEB","T_CACO3","S_CACO3",
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", "."), errors="coerce")

    dom = (df.sort_values(["MU_GLOBAL","SEQ","SHARE"], ascending=[True,True,False])
             .groupby("MU_GLOBAL", as_index=False)
             .first())
    _HWSD_DOM = dom
    return dom

def load_soil_env(lat: float, lon: float) -> dict | None:
    try:
        mu = _mu_from_latlon(lat, lon, raster_path=HWSD_RAS)
        dom = _load_hwsd_dom()
        row = dom.loc[dom["MU_GLOBAL"] == mu]
        if row.empty: return None
        s = row.iloc[0].to_dict()
        s["MU_GLOBAL"] = int(s["MU_GLOBAL"])
        return s
    except Exception:
        return None

# =========================
# 3) İKLİM ve RASTER OKUMA
# =========================
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1; dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def load_climate_nearest(csv_path: Path, lat: float, lon: float) -> dict:
    if not csv_path.exists():
        raise FileNotFoundError(f"İklim CSV bulunamadı: {csv_path}")
    df = pd.read_csv(csv_path)
    if not {"latitude","longitude"}.issubset(df.columns):
        raise ValueError("İklim CSV'de 'latitude' ve 'longitude' sütunları yok.")
    dist = haversine(lat, lon, df["latitude"].values, df["longitude"].values)
    i = int(np.argmin(dist))
    row = df.iloc[i].to_dict()
    row["DISTANCE_KM"] = float(dist[i])
    return row

def sample_raster(path: Path, lon: float, lat: float):
    try:
        import rasterio
        if not path.exists(): return None
        with rasterio.open(path) as ds:
            r, c = ds.index(lon, lat)
            arr = ds.read(1)
            val = arr[r, c]
            if ds.nodata is not None and val == ds.nodata: return None
            return float(val)
    except Exception:
        return None

# =========================
# 4) SUITABILITY (SENİN FORMÜLLERİN)
# =========================
def _clip01(x): x=float(x); return 0.0 if x<0 else (1.0 if x>1 else x)
def _presence(val): return (val is not None) and (str(val).strip() != "")
def _mean_safe(vals): vals=[float(v) for v in vals if _presence(v)]; return sum(vals)/len(vals) if vals else None
def _trapezoid_score(x,a,b,c,d):
    if any(v is None for v in [x,a,b,c,d]): return None
    x=float(x); a,b,c,d = float(a),float(b),float(c),float(d)
    if a>=b or b>c or c>=d: return None
    if x<=a or x>=d: return 0.0
    if b<=x<=c: return 100.0
    if a<x<b: return 100.0*(x-a)/(b-a)
    return 100.0*(d-x)/(d-c)

def suitability_score(crop, env, weights=None, params=None):
    W={'thermal':12,'frost':6,'heat':6,'rad':8,'rh':8,'water':15,'ph':8,'ec':8,'soilphys':5,'taw':4,'esp':3,'caco3':3,'cec':2,'elev':5,'wind':3,'night':2}
    if isinstance(weights, dict): W.update(weights)
    P={'rh_opt':60.0,'rh_span':30.0,'rad_min':1.2,'rad_max':2.8,'frost_band':5.0,'heat_band':5.0,'taw_ref_default':120.0}

    modules, usedW = {}, {}

    Tavg=env.get('T2M_grp2'); Tmin=env.get('T2M_MIN_grp2'); Tmax=env.get('T2M_MAX_grp2')
    tmin_abs=crop.get('tmin_abs'); topt_min=crop.get('topt_min'); topt_max=crop.get('topt_max'); tmax_abs=crop.get('tmax_abs')

    if all(_presence(v) for v in [Tavg,tmin_abs,topt_min,topt_max,tmax_abs]):
        modules['thermal']=_trapezoid_score(Tavg,tmin_abs,topt_min,topt_max,tmax_abs); usedW['thermal']=W['thermal']
    if all(_presence(v) for v in [Tmin,tmin_abs]):
        modules['frost']=100.0 if float(Tmin)>=float(tmin_abs) else max(0.0, 100.0-100.0*(abs(float(tmin_abs)-float(Tmin))/P['frost_band']))
        usedW['frost']=W['frost']
    if all(_presence(v) for v in [Tmax,tmax_abs]):
        modules['heat']=100.0 if float(Tmax)<=float(tmax_abs) else max(0.0, 100.0-100.0*(abs(float(Tmax)-float(tmax_abs))/P['heat_band']))
        usedW['heat']=W['heat']

    R=env.get('ALLSKY_SFC_SW_DWN_grp1')
    if _presence(R):
        modules['rad']=100.0*_clip01((float(R)-P['rad_min'])/max(1e-6,(P['rad_max']-P['rad_min']))); usedW['rad']=W['rad']
    RH=env.get('RH2M_grp2')
    if _presence(RH):
        modules['rh']=100.0*_clip01(1.0-((float(RH)-P['rh_opt'])/P['rh_span'])**2); usedW['rh']=W['rh']

    Pmm=env.get('PRECTOTCORR_grp4'); kc_avg=_mean_safe([crop.get('kc_initial'),crop.get('kc_mid'),crop.get('kc_end')])
    ETc=env.get('ETc'); ET0=env.get('ET0'); AWC=env.get('AWC_MM_PER_M'); Zr=crop.get('root_depth_m')
    if _presence(Pmm) and _presence(kc_avg) and (_presence(ETc) or _presence(ET0)) and _presence(AWC) and _presence(Zr):
        if not _presence(ETc): ETc=float(ET0)*float(kc_avg)
        deficit=max(0.0, float(ETc)-float(Pmm))
        TAW=float(AWC)*float(Zr); denom=max(1.0, TAW/15.0)
        modules['water']=100.0*_clip01(1.0-deficit/denom); usedW['water']=W['water']

    soil_pH=env.get('T_PH_H2O'); pH_min=crop.get('pH_min'); pH_max=crop.get('pH_max')
    if all(_presence(v) for v in [soil_pH,pH_min,pH_max]):
        a=float(pH_min)-0.5; b=float(pH_min); c=float(pH_max); d=float(pH_max)+0.5
        modules['ph']=_trapezoid_score(float(soil_pH),a,b,c,d); usedW['ph']=W['ph']

    soil_EC=env.get('T_ECE'); ec_thr = crop.get('ece_threshold_dSm') if 'ece_threshold_dSm' in crop else crop.get('ece_threshold_dsm')
    if all(_presence(v) for v in [soil_EC,ec_thr]):
        thr=max(0.1,float(ec_thr)); modules['ec']=100.0*_clip01(1.0-float(soil_EC)/thr); usedW['ec']=W['ec']

    tex_ok=(crop.get('texture_ok') or "").lower().replace(" ","")
    tex_ok_set=set([t.strip().lower() for t in tex_ok.split(",") if t.strip()])
    tex_env=(env.get('T_USDA_TEX_DESC') or "").strip().lower().replace(" ","")
    drain_pref=(crop.get('drainage_preference') or "").strip().lower()
    drain_env =(env.get('DRAINAGE_DESC') or "").strip().lower()

    score_tex=None
    if tex_env:
        if tex_env in tex_ok_set: score_tex=100.0
        else:
            neigh={'loam':{'sandy_loam','silt_loam','clay_loam'},'sandy_loam':{'loam'},'silt_loam':{'loam'},'clay_loam':{'loam'},
                   'sandy_clay_loam':{'clay_loam','sandy_loam'},'silty_clay_loam':{'clay_loam','silt_loam'}}
            score_tex=60.0 if any((k in tex_ok_set and tex_env in neigh.get(k,set())) for k in tex_ok_set) else 0.0

    def _norm_drain(s):
        s=s.lower()
        if 'well' in s and 'moderate' not in s: return 'well'
        if 'moderately' in s: return 'moderately well'
        if 'very poorly' in s: return 'very poorly'
        if 'poorly' in s: return 'poorly'
        if 'somewhat' in s: return 'somewhat poorly'
        return None

    score_drain=None
    if drain_pref and drain_env:
        dkey=_norm_drain(drain_env); dmap={'well':100,'moderately well':70,'somewhat poorly':40,'poorly':0,'very poorly':0}
        score_drain=dmap.get(dkey,70.0)

    if score_tex is not None or score_drain is not None:
        parts,wsum=[],0.0
        if score_tex   is not None: parts.append((score_tex,0.6)); wsum+=0.6
        if score_drain is not None: parts.append((score_drain,0.4)); wsum+=0.4
        modules['soilphys']=sum(s*w for s,w in parts)/(wsum if wsum else 1.0); usedW['soilphys']=W['soilphys']

    if _presence(AWC) and _presence(Zr):
        TAW=float(AWC)*float(Zr); ref=float((params or {}).get('taw_ref_default',120.0))
        modules['taw']=100.0*_clip01(TAW/ref); usedW['taw']=W['taw']

    ESP=env.get('T_ESP') or env.get('ESP')
    if _presence(ESP):
        modules['esp']=100.0*_clip01(1.0-float(ESP)/8.0); usedW['esp']=W['esp']

    CACO3=env.get('T_CACO3') or env.get('S_CACO3') or env.get('CACO3')
    if _presence(CACO3):
        modules['caco3']=100.0*_clip01(1.0-float(CACO3)/10.0); usedW['caco3']=W['caco3']

    CEC=env.get('T_CEC_SOIL')
    if _presence(CEC):
        CEC=float(CEC); modules['cec']=40.0 if CEC<8.0 else (70.0 if CEC<12.0 else 100.0); usedW['cec']=W['cec']

    elev=env.get('ELEVATION_M'); elev_min=crop.get('elevation_min'); elev_max=crop.get('elevation_max')
    if _presence(elev):
        e=float(elev)
        if _presence(elev_min) and _presence(elev_max):
            a=float(elev_min)-200.0; b=float(elev_min); c=float(elev_max); d=float(elev_max)+200.0
            modules['elev']=_trapezoid_score(e,a,b,c,d)
        else:
            modules['elev']=100.0 if e<1500 else (70.0 if e<2000 else (40.0 if e<2500 else 0.0))
        usedW['elev']=W['elev']

    WSMAX=env.get('WS2M_MAX_grp3')
    if _presence(WSMAX):
        modules['wind']=100.0*_clip01(1.0-float(WSMAX)/15.0); usedW['wind']=W['wind']

    NL=env.get('NIGHT_LIGHT')
    if _presence(NL):
        modules['night']=100.0*_clip01(float(NL)/5.0); usedW['night']=W['night']

    if not usedW: return {'score': None, 'modules': modules, 'used_weights': usedW}
    wsum=float(sum(usedW.values())); total=0.0
    for k, sc in modules.items():
        if sc is None: continue
        wk=usedW.get(k,0.0)/wsum; total += wk*float(sc)
    return {'score': round(total,2), 'modules': {k:round(v,2) for k,v in modules.items()}, 'used_weights': usedW}

# =========================
# 5) ENV OLUŞTURMA + ÇIKTI
# =========================
def parse_latlon(text: str):
    NUM = r'[-+]?\d+(?:\.\d+)?'
    if text is None: raise ValueError("Boş giriş verildi.")
    s = str(text)
    m = re.search(r'[?&]q=(' + NUM + ')[, ]+(' + NUM + ')', s)
    if not m: m = re.search(r'@(' + NUM + ')[, ]+(' + NUM + ')', s)
    if not m: m = re.search(r'lat[^-+0-9]*(' + NUM + ').*?lon[^-+0-9]*(' + NUM + ')', s, re.IGNORECASE | re.DOTALL)
    if not m:
        nums = re.findall(NUM, s)
        if len(nums) >= 2: lat, lon = float(nums[0]), float(nums[1])
        else: raise ValueError("Lat,lon bulunamadı. Örnek: 41.2397,41.9156 veya Google Maps linki verin.")
    else:
        lat, lon = float(m.group(1)), float(m.group(2))
    if not (-90 <= lat <= 90 and -180 <= lon <= 180):
        raise ValueError(f"Geçersiz aralık: {lat}, {lon}")
    return lat, lon

def build_env(lat: float, lon: float) -> dict:
    env = {}
    clim = load_climate_nearest(CSV_PATH, lat, lon)   # tüm sütunlar
    env.update(clim)                                   # hepsini ekle (örn. *_grpX)

    # rasterlar
    elev  = sample_raster(ELEV_PATH, lon, lat)
    night = sample_raster(LIGHT_PATH, lon, lat)
    if elev  is not None: env['ELEVATION_M'] = elev
    if night is not None: env['NIGHT_LIGHT'] = night

    # toprak
    soil = load_soil_env(lat, lon)
    if soil: env.update(soil)

    # normalize kritik iklim alias'ları (skora girenler)
    def _pick(d,*keys):
        for k in keys:
            if k in d: return d[k]
        return None
    env['T2M_grp2']              = _pick(env,'T2M_grp2','T2M_GRP2','T2M')
    env['T2M_MIN_grp2']          = _pick(env,'T2M_MIN_grp2','T2M_MIN_GRP2','T2M_MIN')
    env['T2M_MAX_grp2']          = _pick(env,'T2M_MAX_grp2','T2M_MAX_GRP2','T2M_MAX')
    env['RH2M_grp2']             = _pick(env,'RH2M_grp2','RH2M_GRP2','RH2M')
    env['ALLSKY_SFC_SW_DWN_grp1']= _pick(env,'ALLSKY_SFC_SW_DWN_grp1','ALLSKY_SFC_SW_DWN_GRP1','ALLSKY_SFC_SW_DWN')
    env['PRECTOTCORR_grp4']      = _pick(env,'PRECTOTCORR_grp4','PRECTOTCORR_GRP4','PRECTOTCORR')
    env['WS2M_MAX_grp3']         = _pick(env,'WS2M_MAX_grp3','WS2M_MAX_GRP3','WS2M_MAX')
    return env

def print_env_ui(env: dict):
    print(f"\n📍 Konum: {format_value(env.get('latitude'))}, {format_value(env.get('longitude'))}\n")
    icons = {"Konum":"📌","İklim":"🌦️","Arazi":"⛰️","Gece Işığı":"🌃","Toprak":"🌱","Özet":"🧾"}

    # Tüm anahtarları kategorize et ve yaz
    keys = sorted(env.keys(), key=lambda k: (CATEGORY_ORDER.index(category_of(k)) if category_of(k) in CATEGORY_ORDER else 99, k))
    current = None
    for k in keys:
        cat = category_of(k)
        # "ham" alias çakışmalarını azaltmak için çok teknik olanları atla
        if k in {"lat","lon"}: 
            continue
        if cat == "Konum" and k.upper() not in {"LATITUDE","LONGITUDE"}:
            continue
        if cat == "Özet" and k != "DISTANCE_KM":
            continue
        # başlık
        title, unit = meta_of(k.replace("_grp1","").replace("_grp2","").replace("_grp3","").replace("_grp4",""))
        # kategori başlığını bas
        if cat != current:
            print(f"{icons.get(cat,'•')} {cat}")
            current = cat
        val = format_value(env[k])
        unit_sfx = f" {unit}" if unit and unit != "-" else ""
        print(f"- {title} ({k}): {val}{unit_sfx}")
    print()

# =========================
# 6) BİTKİLERİ YÜKLE + SKORLAMA
# =========================
def load_crops(crops_csv_path: Path) -> pd.DataFrame:
    if not crops_csv_path.exists():
        raise FileNotFoundError(f"Bitki CSV bulunamadı: {crops_csv_path}")
    df = pd.read_csv(crops_csv_path)
    df.columns = [c.strip().lower() for c in df.columns]
    return df

def row_to_crop_dict(row: pd.Series) -> dict:
    d = row.to_dict()
    if 'ece_threshold_dsm' in d and 'ece_threshold_dSm' not in d:
        d['ece_threshold_dSm'] = d['ece_threshold_dsm']
    if not str(d.get('texture_ok','')).strip():
        d['texture_ok'] = ''
    return d

def weakest_modules(mod_dict, n=2):
    if not mod_dict: return []
    items = [(k,v) for k,v in mod_dict.items() if v is not None]
    if not items: return []
    items.sort(key=lambda x: x[1])
    return [f"{k}:{v:.0f}" for k,v in items[:n]]

def score_and_rank(env: dict, crops_df: pd.DataFrame, top_k: int = 10):
    results = []
    for _, row in crops_df.iterrows():
        crop = row_to_crop_dict(row)
        res = suitability_score(crop, env)
        if res.get('score') is None: 
            continue
        results.append({'crop': crop.get('crop'),
                        'common_name_tr': crop.get('common_name_tr'),
                        'score': res['score'],
                        'modules': res.get('modules', {})})
    if not results:
        print("⚠️ Skor üretilemedi (gerekli çevre/bitki alanları eksik olabilir).")
        return

    results = sorted(results, key=lambda x: x['score'], reverse=True)
    top = results[:max(1, top_k)]

    print("— Ürün Uygunluk Skoru (0–100) —")
    print(f"{'Ürün':<28} {'Skor':>5}  {'Neyi sınırlıyor? (en zayıf 2 modül)':<40}")
    print("-"*80)
    for r in top:
        trname = (r['common_name_tr'] or r['crop'])
        wmods = ", ".join(weakest_modules(r['modules'], n=2)) or "-"
        print(f"{trname:<28} {r['score']:>5.1f}  {wmods:<40}")

    first3 = [ (r['common_name_tr'] or r['crop']) for r in top[:3] ]
    print("\nÖnerilen ilk 3: " + ", ".join(first3))
    print(f"🏆 En uygun ürün: {first3[0]}\n")

# =========================
# 7) REPL — KONUM İSTE + ÇALIŞTIR
# =========================
def ask_for_location():
    try:
        line = input("Konum girin (Google Maps linki YA DA 'lat,lon'):  ").strip()
        if not line:
            raise ValueError("Boş giriş.")
        return line
    except (EOFError, KeyboardInterrupt):
        print("\nİptal edildi."); sys.exit(1)

def main_once():
    q = ask_for_location()
    try:
        lat, lon = parse_latlon(q)
    except Exception as e:
        print(f"⚠️ Konum çözümlenemedi: {e}"); sys.exit(2)

    print(f"\n📍 Konum: {lat:.5f}, {lon:.5f}\nℹ️ Veriler toplanıyor...\n")

    # ENV
    try:
        env = build_env(lat, lon)
    except FileNotFoundError as e:
        print(f"❌ {e}\n👉 Yol(ları) kontrol edin.\n   CROPS_CSV={CROPS_CSV}\n   CLIMATE_CSV={CSV_PATH}\n")
        sys.exit(1)
    except Exception as e:
        print(f"⚠️ ENV oluştururken hata: {type(e).__name__}: {e}"); sys.exit(1)

    # TÜM VERİLERİ UI GİBİ YAZ
    print_env_ui(env)

    # SKORLAMA
    try:
        crops_df = load_crops(CROPS_CSV)
        print("🔎 Uygunluk hesaplanıyor ve sıralanıyor...\n")
        score_and_rank(env, crops_df, top_k=10)
        print("— bitti —")
    except FileNotFoundError as e:
        print(f"❌ {e}\n👉 Bitki CSV yolunu kontrol edin: {CROPS_CSV}")
        sys.exit(1)
    except Exception as e:
        print(f"⚠️ Skorlama sırasında hata: {type(e).__name__}: {e}")
        sys.exit(1)

if __name__ == "__main__":
    if len(sys.argv) > 1:
        q = " ".join(sys.argv[1:])
        try:
            lat, lon = parse_latlon(q)
            print(f"\n📍 Konum: {lat:.5f}, {lon:.5f}\nℹ️ Veriler toplanıyor...\n")
            env = build_env(lat, lon)
            print_env_ui(env)
            crops_df = load_crops(CROPS_CSV)
            print("🔎 Uygunluk hesaplanıyor ve sıralanıyor...\n")
            score_and_rank(env, crops_df, top_k=10)
            print("— bitti —")
        except Exception as e:
            print(f"⚠️ Çalışma sırasında hata: {type(e).__name__}: {e}")
            sys.exit(1)
    else:
        main_once()



📍 Konum: -9.00000, 2.00000
ℹ️ Veriler toplanıyor...


📍 Konum: 37, 27.50

📌 Konum
- Enlem (latitude): 37 °
- Boylam (longitude): 27.50 °
🌦️ İklim
- PAR (tümü) (ALLSKY_SFC_PAR_TOT_grp1): 1.01 MJ/m²/gün
- Kısa dalga (tümü) (ALLSKY_SFC_SW_DWN_grp1): 2.31 kWh/m²/gün
- Bulutluluk (gündüz) (CLOUD_AMT_DAY_grp1): 57.76 %
- Bulutluluk (gece) (CLOUD_AMT_NIGHT_grp1): 55.73 %
- Bulutluluk (CLOUD_AMT_grp1): 56.60 %
- Açık gün sayısı (CLRSKY_DAYS_grp1): 75 gün/ay
- Kısa dalga (açık gök) (CLRSKY_SFC_SW_DWN_grp1): 3.16 kWh/m²/gün
- Uzaklık (iklim pikseli) (DISTANCE_KM): 5764.69 km
- Toplam yağış (düz.) (PRECTOTCORR_grp4): 4.91 mm/gün
- Yüzey basıncı (PS_grp3): 101.04 kPa
- Özgül nem (2 m) (QV2M_grp2): 6.71 g/kg
- Bağıl nem (2 m) (RH2M_grp2): 74.34 %
- Denize indirgenmiş basınç (SLP_grp3): 101.65 kPa
- Çiy noktası (T2MDEW_grp2): 7.78 °C
- Yaş termometre (T2MWET_grp2): 10.05 °C
- Maks. sıcaklık (T2M_MAX_grp2): 19.14 °C
- Min. sıcaklık (T2M_MIN_grp2): 0.76 °C
- Günlük sıcaklık aralığı (T2M_RANGE_grp2): 